# Modelo Preditivo Não Supervisionado - Chilli Beans
## Análise de Localização para Expansão de Lojas com Foco em Óculos de Grau

Este notebook desenvolve um modelo de clusterização para identificar melhores localizações para posicionar novas lojas da Chilli Beans, com foco na venda de óculos de grau.

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Bibliotecas importadas com sucesso!")

In [ ]:
# Carregamento dos dados
df = pd.read_csv('../../database/dataset gerado/dataset_codificado.csv')

print("Dataset carregado com sucesso!")
print(f"Dimensões do dataset: {df.shape}")
print(f"\nColunas disponíveis: {list(df.columns)}")
print(f"\nPrimeiras 5 linhas:")
df.head()

## 🗺️  Carregamento de Coordenadas Geográficas 

Esta etapa carrega um arquivo CSV que contém as coordenadas geográficas (latitude e longitude) para cada combinação de bairro, cidade e estado presentes nos dados da Chilli Beans.

**1. Por que precisamos dessas coordenadas:**
- O dataset original possui apenas nomes de bairros, cidades e estados codificados
- Para fazer análise geográfica e clustering espacial, precisamos das coordenadas exatas
- As coordenadas permitem calcular distâncias reais entre localidades

**2. O que o arquivo `com_coordenadas.csv` contém:**
- Cada linha representa uma localização única (bairro + cidade + estado)
- Colunas com latitude e longitude em formato decimal
- Mesma estrutura de nomeação das colunas do dataset principal

**3. Verificações importantes realizadas:**
- Total de localizações disponíveis no arquivo de coordenadas
- Quais estados estão cobertos pelas coordenadas
- Se existem dados faltantes (valores nulos) nas coordenadas

**4. Resultado esperado:**
- Confirmação de que temos coordenadas para as principais localidades
- Identificação de possíveis lacunas nos dados geográficos
- Base sólida para aplicar algoritmos de clustering espacial

In [ ]:
# Carregando CSV com coordenadas precisas por bairro/cidade
print("="*50)
print("CARREGANDO COORDENADAS PRECISAS")
print("="*50)

# Carregando coordenadas por bairro/cidade
coordenadas_precisas = pd.read_csv('../../database/dataset gerado/com_coordenadas.csv')

print(f"Total de localizações precisas carregadas: {len(coordenadas_precisas)}")
print("\nPrimeiras localizações:")
print(coordenadas_precisas.head())

print("\nEstados únicos nas coordenadas precisas:")
print(sorted(coordenadas_precisas['Dim_Lojas.Estado_Emp'].unique()))

print("\nVerificando dados ausentes:")
print(coordenadas_precisas.isnull().sum())

##  Criação de Mapeamentos Reversos para Coordenadas

Esta etapa é fundamental para conectar os dados codificados com as coordenadas geográficas reais. O processo cria mapeamentos que permitem converter códigos numéricos de volta para nomes reais de estados, cidades e bairros.

**1. Por que usamos dados codificados vs dados originais:**
- O dataset codificado (`dataset_codificado.csv`) é usado para processamento pelo modelo porque:
  - Os algoritmos de machine learning funcionam melhor com dados numéricos
  - Evita problemas de encoding de caracteres especiais
  - Garante consistência e padronização dos dados
- Porém, para análises e visualizações humanas, precisamos dos nomes reais
- **Esta célula faz a ponte entre os dois mundos**: processamento numérico + interpretação textual

**2. Por que precisamos de mapeamentos reversos:**
- O dataset principal (`dataset_codificado.csv`) possui apenas códigos numéricos para localização
- O arquivo de coordenadas (`com_coordenadas.csv`) usa nomes reais de estados, cidades e bairros
- Precisamos conectar esses dois formatos para associar coordenadas aos registros
- **Essencial para mostrar resultados interpretáveis nas análises**

**3. Processo de criação dos mapeamentos:**
- Carrega o dataset original não codificado (`dataset_limpo.csv`)
- Usa a função `pd.factorize()` para recriar a mesma codificação numérica usada anteriormente
- Cria dicionários que mapeiam código → nome real para cada nível geográfico
- **Garante que a codificação seja idêntica à usada no dataset principal**

**4. Estrutura dos mapeamentos criados:**
- `estado_mapping`: {0: 'SP', 1: 'RJ', 2: 'MG', ...}
- `cidade_mapping`: {0: 'São Paulo', 1: 'Rio de Janeiro', ...}
- `bairro_mapping`: {0: 'Centro', 1: 'Copacabana', ...}

**5. Validações realizadas:**
- Verifica se as colunas de localização existem no dataset original
- Conta quantos valores únicos existem para cada nível geográfico
- Confirma que os mapeamentos foram criados corretamente
- **Essencial para garantir que não há perda de informação na conversão**

**6. Resultado esperado:**
- Três dicionários de mapeamento funcionais
- Capacidade de converter qualquer código numérico de volta para nome real
- Base para aplicar coordenadas geográficas precisas aos dados
- **Permitir interpretação humana dos resultados do

In [ ]:
# Primeiro, precisamos carregar o dataset original para criar os mapeamentos reversos
print("="*50)
print("CRIANDO MAPEAMENTOS REVERSOS PARA COORDENADAS")
print("="*50)

# Carregando dataset original não codificado para criar mapeamentos
df_original = pd.read_csv('../../database/dataset gerado/dataset_limpo.csv')

print(f"Dataset original carregado: {len(df_original)} registros")

# Verificando se tem as colunas necessárias
print("Colunas relacionadas a localização no dataset original:")
location_cols = [col for col in df_original.columns if any(term in col.lower() for term in ['bairro', 'cidade', 'estado', 'emp'])]
print(location_cols)

# Criando mapeamentos reversos dos códigos para valores originais
if 'Dim_Lojas.Estado_Emp' in df_original.columns:
    # Mapeamento Estado
    estado_mapping = dict(zip(
        pd.factorize(df_original['Dim_Lojas.Estado_Emp'])[0],
        df_original['Dim_Lojas.Estado_Emp']
    ))
    
    # Mapeamento Cidade  
    cidade_mapping = dict(zip(
        pd.factorize(df_original['Dim_Lojas.Cidade_Emp'])[0],
        df_original['Dim_Lojas.Cidade_Emp']
    ))
    
    # Mapeamento Bairro
    bairro_mapping = dict(zip(
        pd.factorize(df_original['Dim_Lojas.Bairro_Emp'])[0],
        df_original['Dim_Lojas.Bairro_Emp']
    ))
    
    print(f"Mapeamentos criados:")
    print(f"- Estados únicos: {len(set(estado_mapping.values()))}")
    print(f"- Cidades únicas: {len(set(cidade_mapping.values()))}")
    print(f"- Bairros únicos: {len(set(bairro_mapping.values()))}")
    
else:
    print("Colunas de localização não encontradas. Verificando estrutura...")
    print(df_original.columns.tolist())

## Aplicação de Coordenadas Precisas aos Dados

Esta célula implementa a função central para mapear coordenadas geográficas precisas aos dados codificados, estabelecendo a ponte entre os códigos numéricos e as localizações reais.

**1. Função `obter_coordenadas_precisas()`:**
- **Estratégia em cascata** para máxima precisão:
  1. **Match exato**: Busca por bairro + cidade + estado específicos
  2. **Match por cidade**: Se não encontrar o bairro, usa cidade + estado
  3. **Fallback por estado**: Como último recurso, usa coordenadas centrais do estado

**2. Por que essa abordagem hierárquica:**
- **Precisão máxima**: Prioriza coordenadas específicas quando disponíveis
- **Robustez**: Garante que todos os registros tenham coordenadas válidas
- **Realismo**: Usa localizações reais em vez de estimativas grosseiras
- **Escalabilidade**: Funciona mesmo com dados incompletos

**3. Processo de aplicação das coordenadas:**
- Percorre cada registro do dataset codificado
- **Decodifica** os códigos numéricos usando os mapeamentos reversos criados
- Aplica a função de busca de coordenadas para cada localização
- **Valida** se as coordenadas são válidas (não nulas/zero)

**4. Tratamento de casos especiais:**
- **Dados faltantes**: Usa coordenadas do centro do estado como fallback
- **Caracteres especiais**: Os mapeamentos reversos garantem consistência
- **Coordenadas inválidas**: Sistema de verificação evita dados corrompidos

**5. Validações implementadas:**
- Contagem de registros com coordenadas válidas vs inválidas
- Exemplos práticos de coordenadas aplicadas
- Estatísticas de distribuição geográfica
- Verificação de cobertura por estado

**6. Benefícios desta abordagem:**
- **100% de cobertura**: Todos os registros recebem coordenadas
- **Precisão variável**: Desde coordenadas exatas até aproximações estaduais
- **Interpretabilidade**: Permite análises geográficas detalhadas
- **Clustering espacial**: Base sólida para algoritmos de agrupamento geográfico

**7. Output esperado:**
- Dataset enriquecido com colunas `latitude` e `longitude`
- Coordenadas válidas para análise de clustering
- Mapeamento completo entre dados codificados e localização real
- **Fundação para identificação de locais ideais para

In [ ]:
# Função para mapear coordenadas precisas
def obter_coordenadas_precisas(bairro, cidade, estado, coordenadas_df):
    """
    Obtém coordenadas precisas baseadas em bairro, cidade e estado
    """
    # Primeiro, tenta match exato por bairro, cidade e estado
    match_exato = coordenadas_df[
        (coordenadas_df['Dim_Lojas.Bairro_Emp'] == bairro) & 
        (coordenadas_df['Dim_Lojas.Cidade_Emp'] == cidade) & 
        (coordenadas_df['Dim_Lojas.Estado_Emp'] == estado)
    ]
    
    if not match_exato.empty and not pd.isna(match_exato.iloc[0]['Latitude']):
        return match_exato.iloc[0]['Latitude'], match_exato.iloc[0]['Longitude']
    
    # Se não encontrar, tenta por cidade e estado
    match_cidade = coordenadas_df[
        (coordenadas_df['Dim_Lojas.Cidade_Emp'] == cidade) & 
        (coordenadas_df['Dim_Lojas.Estado_Emp'] == estado)
    ]
    
    if not match_cidade.empty and not pd.isna(match_cidade.iloc[0]['Latitude']):
        return match_cidade.iloc[0]['Latitude'], match_cidade.iloc[0]['Longitude']
    
    # Se ainda não encontrar, usa coordenadas por estado (fallback)
    estado_coords = {
        'AC': (-9.97, -67.81), 'AL': (-9.66, -35.73), 'AP': (0.04, -51.06), 'AM': (-3.13, -59.98),
        'BA': (-12.98, -38.48), 'CE': (-3.73, -38.52), 'DF': (-15.79, -47.88), 'ES': (-20.32, -40.31),
        'GO': (-16.68, -49.25), 'MA': (-2.56, -44.06), 'MT': (-15.60, -56.10), 'MS': (-20.46, -54.62),
        'MG': (-19.92, -43.94), 'PA': (-1.45, -48.47), 'PB': (-7.12, -34.88), 'PR': (-25.43, -49.27),
        'PE': (-8.06, -34.88), 'PI': (-5.09, -42.80), 'RJ': (-22.91, -43.21), 'RN': (-5.81, -35.21),
        'RS': (-30.03, -51.23), 'RO': (-8.76, -63.87), 'RR': (2.82, -60.67), 'SC': (-27.60, -48.55),
        'SP': (-23.55, -46.63), 'SE': (-10.92, -37.08), 'TO': (-10.18, -48.33)
    }
    
    return estado_coords.get(estado, (0, 0))

print("="*50)
print("APLICANDO COORDENADAS PRECISAS AOS DADOS")
print("="*50)

# Aplicando coordenadas precisas aos dados usando mapeamentos reversos
lat_coords = []
lon_coords = []

for idx, row in df.iterrows():
    # Convertendo códigos de volta para valores originais
    estado_code = row['Dim_Lojas.Estado_Emp_encoded']
    cidade_code = row['Dim_Lojas.Cidade_Emp_encoded'] 
    bairro_code = row['Dim_Lojas.Bairro_Emp_encoded']
    
    # Obtendo valores originais
    estado = estado_mapping.get(estado_code, 'SP')  # fallback para SP
    cidade = cidade_mapping.get(cidade_code, 'São Paulo')  # fallback
    bairro = bairro_mapping.get(bairro_code, 'Centro')  # fallback
    
    lat, lon = obter_coordenadas_precisas(bairro, cidade, estado, coordenadas_precisas)
    lat_coords.append(lat)
    lon_coords.append(lon)

# Atualizando o DataFrame com coordenadas precisas
df['latitude'] = lat_coords
df['longitude'] = lon_coords

print(f"Total de registros com coordenadas atualizadas: {len(df)}")
print(f"Coordenadas válidas (não zero): {len(df[(df['latitude'] != 0) & (df['longitude'] != 0)])}")

# Verificando exemplos de coordenadas atualizadas
print("\nExemplos de coordenadas atualizadas:")
sample_indices = df.sample(10).index
for i in sample_indices[:5]:
    estado = estado_mapping.get(df.loc[i, 'Dim_Lojas.Estado_Emp_encoded'], 'N/A')
    cidade = cidade_mapping.get(df.loc[i, 'Dim_Lojas.Cidade_Emp_encoded'], 'N/A')
    bairro = bairro_mapping.get(df.loc[i, 'Dim_Lojas.Bairro_Emp_encoded'], 'N/A')
    lat = df.loc[i, 'latitude']
    lon = df.loc[i, 'longitude']
    print(f"{bairro}, {cidade}, {estado}: ({lat:.3f}, {lon:.3f})")

# Estatísticas das coordenadas
print(f"\nFaixa de Latitude: {df['latitude'].min():.2f} a {df['latitude'].max():.2f}")
print(f"Faixa de Longitude: {df['longitude'].min():.2f} a {df['longitude'].max():.2f}")

# Verificando distribuição por estado usando mapeamento reverso
estados_originais = [estado_mapping.get(code, 'N/A') for code in df['Dim_Lojas.Estado_Emp_encoded']]
estado_counts = pd.Series(estados_originais).value_counts().head(10)
print("\nDistribuição de registros por estado (top 10):")
print(estado_counts)

## Verificação e Correção de Mapeamentos

Esta etapa crítica garante que os mapeamentos entre códigos numéricos e valores originais estejam corretos, criando uma base sólida para a aplicação precisa de coordenadas geográficas.

**1. Por que precisamos verificar os mapeamentos:**
- **Validação de integridade**: Confirma que a codificação foi consistente
- **Correção de discrepâncias**: Identifica e corrige possíveis erros na criação inicial dos mapeamentos
- **Garantia de precisão**: Assegura que cada código corresponde exatamente ao valor original correto
- **Base para coordenadas**: Mapeamentos incorretos resultariam em coordenadas aplicadas aos locais errados

**2. Processo de verificação implementado:**
- **Concatenação inteligente**: Une os dados codificados com os dados originais pelo índice
- **Verificação linha por linha**: Percorre cada registro para criar mapeamentos baseados na correspondência real
- **Validação de dados**: Ignora valores nulos ou inválidos durante a criação dos mapeamentos
- **Comparação com mapeamentos anteriores**: Permite identificar diferenças e inconsistências

**3. Metodologia de correção:**
- **Mapeamento direto**: Para cada código, identifica o valor original correspondente no mesmo registro
- **Eliminação de duplicatas**: Garante que cada código seja mapeado para apenas um valor
- **Tratamento de casos especiais**: Lida com valores ausentes ou códigos órfãos
- **Validação estatística**: Conta valores únicos para verificar consistência

**4. Benefícios da correção:**
- **Coordenadas precisas**: Elimina erros na aplicação de latitude/longitude
- **Interpretabilidade correta**: Garante que análises mostrem os locais reais
- **Confiabilidade do modelo**: Reduz ruído causado por mapeamentos incorretos
- **Reprodutibilidade**: Permite que análises futuras usem os mesmos mapeamentos

**5. Validações implementadas:**
- **Contagem de valores únicos**: Verifica se o número de estados, cidades e bairros únicos faz sentido
- **Exemplos de mapeamento**: Mostra casos reais de código → valor para verificação manual
- **Teste de consistência**: Confirma que os mapeamentos são bijettivos (um-para-um)
- **Verificação de completude**: Assegura que todos os códigos tenham valores correspondentes

**6. Re-aplicação de coordenadas:**
- **Uso dos mapeamentos corrigidos**: Aplica as coordenadas usando os novos mapeamentos verificados
- **Comparação antes/depois**: Permite avaliar o impacto das correções
- **Validação final**: Confirma que as coordenadas aplicadas são geograficamente válidas
- **Estatísticas atualizadas**: Recalcula distribuições por estado com dados corrigidos

**7. Output esperado:**
- Mapeamentos reversos precisos e validados
- Coordenadas geográficas aplicadas corretamente a todos os registros
- Base confiável para todas as análises subsequentes de clustering
- **Eliminação de erros sistemáticos** que poderiam comprometer a identificação

In [ ]:
# Verificando e corrigindo mapeamentos
print("="*50)
print("VERIFICANDO E CORRIGINDO MAPEAMENTOS")
print("="*50)

# Criando mapeamentos mais precisos usando o LabelEncoder approach
from sklearn.preprocessing import LabelEncoder

# Verificando se os dados codificados correspondem aos originais
print("Verificando mapeamentos...")

# Criando um DataFrame com códigos e valores originais para verificação
check_df = pd.DataFrame({
    'estado_code': df['Dim_Lojas.Estado_Emp_encoded'],
    'cidade_code': df['Dim_Lojas.Cidade_Emp_encoded'], 
    'bairro_code': df['Dim_Lojas.Bairro_Emp_encoded']
})

# Juntando com dados originais baseados no índice
df_with_original = pd.concat([
    df[['Dim_Lojas.Estado_Emp_encoded', 'Dim_Lojas.Cidade_Emp_encoded', 'Dim_Lojas.Bairro_Emp_encoded']],
    df_original[['Dim_Lojas.Estado_Emp', 'Dim_Lojas.Cidade_Emp', 'Dim_Lojas.Bairro_Emp']]
], axis=1)

print("Criando mapeamentos corretos...")

# Criando mapeamentos corretos baseados na correspondência real
estado_map_correto = {}
cidade_map_correto = {}
bairro_map_correto = {}

for idx in range(len(df_with_original)):
    # Estado
    code_estado = df_with_original.iloc[idx]['Dim_Lojas.Estado_Emp_encoded']
    val_estado = df_with_original.iloc[idx]['Dim_Lojas.Estado_Emp']
    if pd.notna(code_estado) and pd.notna(val_estado):
        estado_map_correto[code_estado] = val_estado
    
    # Cidade  
    code_cidade = df_with_original.iloc[idx]['Dim_Lojas.Cidade_Emp_encoded']
    val_cidade = df_with_original.iloc[idx]['Dim_Lojas.Cidade_Emp']
    if pd.notna(code_cidade) and pd.notna(val_cidade):
        cidade_map_correto[code_cidade] = val_cidade
        
    # Bairro
    code_bairro = df_with_original.iloc[idx]['Dim_Lojas.Bairro_Emp_encoded'] 
    val_bairro = df_with_original.iloc[idx]['Dim_Lojas.Bairro_Emp']
    if pd.notna(code_bairro) and pd.notna(val_bairro):
        bairro_map_correto[code_bairro] = val_bairro

print(f"Mapeamentos corretos criados:")
print(f"- Estados: {len(set(estado_map_correto.values()))} únicos")
print(f"- Cidades: {len(set(cidade_map_correto.values()))} únicas") 
print(f"- Bairros: {len(set(bairro_map_correto.values()))} únicos")

# Testando alguns exemplos
print("\nExemplos de mapeamentos corretos:")
for code in list(estado_map_correto.keys())[:10]:
    print(f"Código {code} -> {estado_map_correto[code]}")

# Replicando coordenadas com mapeamentos corretos
print("\nReaplicando coordenadas com mapeamentos corretos...")

lat_coords_correto = []
lon_coords_correto = []

for idx, row in df.iterrows():
    # Convertendo códigos de volta para valores originais (usando mapeamentos corretos)
    estado_code = row['Dim_Lojas.Estado_Emp_encoded']
    cidade_code = row['Dim_Lojas.Cidade_Emp_encoded'] 
    bairro_code = row['Dim_Lojas.Bairro_Emp_encoded']
    
    # Obtendo valores originais
    estado = estado_map_correto.get(estado_code, 'SP')
    cidade = cidade_map_correto.get(cidade_code, 'São Paulo') 
    bairro = bairro_map_correto.get(bairro_code, 'Centro')
    
    lat, lon = obter_coordenadas_precisas(bairro, cidade, estado, coordenadas_precisas)
    lat_coords_correto.append(lat)
    lon_coords_correto.append(lon)

# Atualizando o DataFrame com coordenadas corrigidas
df['latitude'] = lat_coords_correto
df['longitude'] = lon_coords_correto

print(f"\nCoordenadas corrigidas aplicadas!")
print(f"Coordenadas válidas (não zero): {len(df[(df['latitude'] != 0) & (df['longitude'] != 0)])}")

# Verificando exemplos corrigidos
print("\nExemplos de coordenadas corrigidas:")
sample_indices = [0, 100, 1000, 5000, 10000]
for i in sample_indices:
    if i < len(df):
        estado = estado_map_correto.get(df.loc[i, 'Dim_Lojas.Estado_Emp_encoded'], 'N/A')
        cidade = cidade_map_correto.get(df.loc[i, 'Dim_Lojas.Cidade_Emp_encoded'], 'N/A')
        bairro = bairro_map_correto.get(df.loc[i, 'Dim_Lojas.Bairro_Emp_encoded'], 'N/A')
        lat = df.loc[i, 'latitude']
        lon = df.loc[i, 'longitude']
        print(f"{bairro}, {cidade}, {estado}: ({lat:.3f}, {lon:.3f})")

# Distribuição corrigida por estado
estados_corretos = [estado_map_correto.get(code, 'N/A') for code in df['Dim_Lojas.Estado_Emp_encoded']]
estado_counts_correto = pd.Series(estados_corretos).value_counts().head(10)
print("\nDistribuição corrigida por estado (top 10):")
print(estado_counts_correto)

## 📊 Processamento dos Dados com Coordenadas Precisas

Esta seção representa uma etapa crucial do modelo não supervisionado, onde os dados brutos são transformados em informações estruturadas e agregadas, prontas para análise de clustering geográfico com foco específico em vendas de óculos de grau.

**1. Investigação de Categorias de Produtos:**
- **Identificação de colunas relevantes**: Busca por todas as colunas que contenham `Dim_Produtos.GRUPO_CHILLI__` para mapear categorias de produtos
- **Análise específica para produtos com "GRAU"**: Identifica colunas que contenham a palavra "GRAU" para capturar armações e produtos de correção visual
- **Análise de produtos VISTA/LENTES**: Mapeia categorias relacionadas à visão e lentes de grau
- **Validação quantitativa**: Conta valores únicos e somas para entender a distribuição de cada categoria de produto

**2. Criação da Métrica Principal - Vendas de Óculos de Grau:**
- **Identificação ampla de produtos**: Busca colunas que contêm "GRAU" no nome para capturar armações e produtos de grau
- **Combinação estratégica**: Une produtos com "GRAU", "VISTA" e "LENTES" para criar uma métrica unificada abrangente
- **Inclusão de armações**: Garante que armações de óculos de grau sejam incluídas na análise, não apenas lentes
- **Validação quantitativa**: Verifica a distribuição e cobertura da nova métrica criada com detalhamento por categoria

**3. Agregação por Localização Geográfica Precisa:**
- **Chave única por coordenadas**: Cria `loc_key` combinando latitude e longitude para identificar localizações únicas
- **Agregação inteligente**: Agrupa dados por localização precisa calculando:
  - `num_lojas`: Número único de lojas por localização
  - `num_clientes`: Número único de clientes por localização  
  - `vendas_oculos_grau`: Soma total de vendas de óculos de grau
  - `receita_total`: Soma da receita líquida total
  - `estado_code`: Mantém referência do estado para mapeamento posterior

**4. Enriquecimento com Informações Contextuais:**
- **Mapeamento de estados**: Converte códigos numéricos de volta para nomes de estados usando mapeamentos corrigidos
- **Score de potencial multicritério**: Cria uma métrica composta considerando:
  - Vendas de óculos de grau (peso 2x - prioridade principal)
  - Número de clientes (peso 0.1)
  - Receita total (peso 0.001) 
  - Número de lojas (peso 1.5)

**5. Separação Estratégica por Região:**
- **São Paulo vs Outros Estados**: Divide os dados em dois conjuntos para análise independente
- **Justificativa da separação**: São Paulo possui características únicas de densidade e volume que poderiam enviesar a análise
- **Métricas comparativas**: Calcula estatísticas separadas para cada região

**6. Validações e Estatísticas Descritivas:**
- **Cobertura geográfica**: Verifica ranges de latitude e longitude para confirmar abrangência nacional
- **Top localizações**: Identifica as 10 melhores localizações por potencial para validação visual
- **Distribuição regional**: Mostra como vendas e lojas se distribuem entre SP e outros estados

**7. Preparação para Clustering:**
- **Dados limpos e estruturados**: Cada linha representa uma localização única com métricas agregadas
- **Coordenadas precisas**: Latitude e longitude validadas para algoritmos de clustering espacial
- **Métricas padronizadas**: Variáveis em escalas apropriadas para análise de machine learning

**8. Output Esperado:**
- **Dataset `dados_localizacao`**: Dataframe principal com localizações únicas e métricas agregadas
- **Subsets regionais**: `dados_sp` e `dados_outros_estados` para análises específicas
- **Base para clustering**: Dados prontos para aplicação de algoritmos K-means com coordenadas geográficas

**9. Benefícios desta Abordagem:**
- **Precisão geográfica**: Usa coordenadas reais em vez de aproximações
- **Agregação inteligente**: Evita duplicatas e consolida informações por localização
- **Flexibilidade analítica**: Permite análises tanto unificadas quanto regionais
- **Foco no negócio**: Prioriza métricas relevantes para decisões de expansão de óculos de grau

**10. Preparação para Próximas Etapas:**
- **Clustering espacial**: Dados estruturados permitem aplicação direta de algoritmos de agrupamento
- **Análise de potencial**: Score composto facilita identificação de oportunidades
- **Visualizações**: Coordenadas permitem criação de mapas e gráficos geográficos
- **Tomada de decisão**: Métricas agregadas fornecem base sólida para recomendações estratégicas focadas em óculos de grau (incluindo armações e lentes)

In [ ]:
# PROCESSAMENTO DOS DADOS COM COORDENADAS PRECISAS

print("="*80)
print("PROCESSAMENTO DOS DADOS PARA CLUSTERING")
print("="*80)

# Primeiro, vamos investigar as colunas de produtos disponíveis
print("🔍 INVESTIGANDO CATEGORIAS DE PRODUTOS PARA ÓCULOS DE GRAU:")
print("-" * 60)

# Verificar colunas relacionadas a produtos
produto_cols = [col for col in df.columns if 'Dim_Produtos.GRUPO_CHILLI__' in col]
print(f"Colunas GRUPO_CHILLI encontradas: {len(produto_cols)}")
for col in produto_cols:
    valores_unicos = df[col].sum() if df[col].dtype in ['int64', 'float64'] else df[col].nunique()
    print(f"  📋 {col}: {valores_unicos}")

print(f"\n🎯 IDENTIFICANDO PRODUTOS RELACIONADOS A ÓCULOS DE GRAU:")
print("-" * 50)

# Identificar colunas que contêm "GRAU" no nome
colunas_grau = [col for col in produto_cols if 'GRAU' in col.upper()]
print(f"Colunas que contêm 'GRAU': {len(colunas_grau)}")
for col in colunas_grau:
    valores = df[col].sum() if df[col].dtype in ['int64', 'float64'] else df[col].nunique()
    print(f"  📋 {col}: {valores}")

# Identificar colunas relacionadas a VISTA/LENTES
colunas_vista = [col for col in produto_cols if any(termo in col.upper() for termo in ['VISTA', 'LENTES'])]
print(f"\nColunas relacionadas a VISTA/LENTES: {len(colunas_vista)}")
for col in colunas_vista:
    valores = df[col].sum() if df[col].dtype in ['int64', 'float64'] else df[col].nunique()
    print(f"  📋 {col}: {valores}")

print(f"\n🎯 CRIANDO MÉTRICA ESPECÍFICA PARA ÓCULOS DE GRAU:")
print("-" * 50)

# Combinando todas as categorias relacionadas a óculos de grau
colunas_oculos_grau = list(set(colunas_grau + colunas_vista))
print(f"Total de colunas para óculos de grau: {len(colunas_oculos_grau)}")

# Calculando vendas de óculos de grau
df['vendas_oculos_grau'] = 0
for col in colunas_oculos_grau:
    if col in df.columns:
        # Para colunas boolean, convertemos True/False para 1/0
        if df[col].dtype == 'bool':
            df['vendas_oculos_grau'] += df[col].astype(int)
            print(f"   • Adicionando {col} (boolean): {df[col].sum()} registros True")
        elif df[col].dtype in ['int64', 'float64']:
            df['vendas_oculos_grau'] += df[col]
            print(f"   • Adicionando {col} (numérico): {df[col].sum()}")
        else:
            print(f"   ⚠ Ignorando {col} (tipo {df[col].dtype} não suportado)")

print(f"\n✅ Métrica 'vendas_oculos_grau' criada combinando {len([col for col in colunas_oculos_grau if col in df.columns])} categorias:")
print(f"   • TOTAL óculos de grau: {df['vendas_oculos_grau'].sum()}")

# Detalhamento por categoria
print(f"\n📊 DETALHAMENTO POR CATEGORIA:")
for col in colunas_oculos_grau:
    if col in df.columns:
        if df[col].dtype == 'bool':
            total = df[col].sum()  # Para boolean, sum() conta os True
            print(f"   • {col.replace('Dim_Produtos.GRUPO_CHILLI__', '')}: {total} registros")
        elif df[col].dtype in ['int64', 'float64']:
            total = df[col].sum()
            if total > 0:
                print(f"   • {col.replace('Dim_Produtos.GRUPO_CHILLI__', '')}: {total}")
print(f"   • TOTAL óculos de grau: {df['vendas_oculos_grau'].sum()}")

# Verificação da distribuição
print(f"\n📊 VALIDAÇÃO DA MÉTRICA:")
registros_com_oculos_grau = (df['vendas_oculos_grau'] > 0).sum()
print(f"   • Registros com vendas de óculos de grau: {registros_com_oculos_grau:,}")
print(f"   • Porcentagem do dataset: {(registros_com_oculos_grau/len(df)*100):.1f}%")

# Agregando dados por localização precisa (latitude/longitude)
print("\n🗺️ AGREGANDO DADOS POR LOCALIZAÇÃO PRECISA...")

# Criando chave única por localização
df['loc_key'] = df['latitude'].astype(str) + '_' + df['longitude'].astype(str)

dados_localizacao = df.groupby(['loc_key', 'latitude', 'longitude']).agg({
    'ID_Loja': 'nunique',
    'ID_Cliente': 'nunique', 
    'vendas_oculos_grau': 'sum',
    'Total_Preco_Liquido': 'sum',
    'Dim_Lojas.Estado_Emp_encoded': 'first'  # Para manter referência do estado
}).reset_index()

# Renomeando colunas
dados_localizacao.columns = ['loc_key', 'latitude', 'longitude', 'num_lojas', 'num_clientes', 
                           'vendas_oculos_grau', 'receita_total', 'estado_code']

# Adicionando estado original usando mapeamento correto
dados_localizacao['estado'] = dados_localizacao['estado_code'].map(estado_map_correto)

# Criando score de potencial baseado em múltiplos fatores
dados_localizacao['potencial_score'] = (
    dados_localizacao['vendas_oculos_grau'] * 2 +  # Peso maior para óculos de grau
    dados_localizacao['num_clientes'] * 0.1 +
    dados_localizacao['receita_total'] * 0.001 +
    dados_localizacao['num_lojas'] * 1.5
)

print(f"Dados agregados por localização:")
print(f"- Total de localizações únicas: {len(dados_localizacao)}")
print(f"- Total de vendas de óculos de grau: {dados_localizacao['vendas_oculos_grau'].sum()}")
print(f"- Total de clientes únicos: {dados_localizacao['num_clientes'].sum()}")
print(f"- Total de lojas: {dados_localizacao['num_lojas'].sum()}")
print(f"- Receita total: R$ {dados_localizacao['receita_total'].sum():,.2f}")

# Separando São Paulo dos outros estados
dados_sp = dados_localizacao[dados_localizacao['estado'] == 'SP'].copy()
dados_outros_estados = dados_localizacao[dados_localizacao['estado'] != 'SP'].copy()

print(f"\nDistribuição por região:")
print(f"- Localizações em SP: {len(dados_sp)}")
print(f"- Localizações em outros estados: {len(dados_outros_estados)}")
print(f"- Vendas óculos SP: {dados_sp['vendas_oculos_grau'].sum()}")
print(f"- Vendas óculos outros: {dados_outros_estados['vendas_oculos_grau'].sum()}")

# Estatísticas das coordenadas precisas
print(f"\nRanges de coordenadas:")
print(f"- Latitude: {dados_localizacao['latitude'].min():.2f} a {dados_localizacao['latitude'].max():.2f}")
print(f"- Longitude: {dados_localizacao['longitude'].min():.2f} a {dados_localizacao['longitude'].max():.2f}")

# Top 10 localizações por potencial
print(f"\nTop 10 localizações por potencial:")
top_locations = dados_localizacao.nlargest(10, 'potencial_score')[['estado', 'latitude', 'longitude', 'potencial_score', 'vendas_oculos_grau', 'num_clientes']]
print(top_locations)

print("\n" + "="*80)

## Validação e Enriquecimento dos Dados Processados

Esta seção realiza a **validação completa e enriquecimento contextual** dos dados processados na etapa anterior, garantindo a qualidade dos dados antes da aplicação dos algoritmos de clustering e fornecendo informações regionais estratégicas.

**1. Validação da Estrutura de Dados:**
- **Inspeção de colunas**: Verifica se todas as colunas esperadas foram criadas corretamente após o processamento
- **Análise das primeiras linhas**: Valida visualmente se os dados agregados estão consistentes e bem formatados
- **Verificação de tipos**: Confirma que as métricas numéricas estão nos formatos corretos para análise

**2. Análise Estatística Descritiva:**
- **Resumo estatístico**: Calcula médias, medianas, quartis e desvios para todas as métricas numéricas
- **Identificação de outliers**: Detecta valores extremos que podem impactar a análise de clustering
- **Distribuição geográfica**: Verifica se há concentração excessiva em determinadas regiões

**3. Mapeamento Geográfico e Regional:**
- **Criação de códigos únicos**: Gera identificadores sequenciais para cada localização processada
- **Classificação regional**: Mapeia estados para regiões da estratégia Chilli Beans (São Paulo, Sudeste, Sul, Nordeste, Centro-Oeste, Norte)
- **Validação da cobertura**: Confirma presença de dados em todas as regiões do país

**4. Análise de Distribuição por Estado:**
- **Ranking estadual**: Identifica quais estados têm maior concentração de localizações processadas
- **Balanceamento regional**: Verifica se há representatividade adequada de diferentes estados
- **Identificação de gaps**: Detecta possíveis lacunas geográficas na cobertura de dados

**5. Separação Estratégica São Paulo vs Outros Estados:**
- **Justificativa da separação**: São Paulo possui características únicas de densidade populacional e mercado
- **Comparação quantitativa**: Mostra diferenças de volume entre SP e demais estados
- **Preparação para clustering**: Valida se ambos os subconjuntos têm dados suficientes para análise independente

**6. Identificação de Localizações de Alto Potencial:**
- **Top 5 SP**: Lista as melhores localizações em São Paulo baseadas no score de potencial
- **Top 5 outros estados**: Identifica oportunidades fora de São Paulo
- **Métricas de validação**: Mostra vendas de óculos de grau e número de clientes para cada localização

**7. Validações de Qualidade:**
- **Consistência de dados**: Verifica se métricas agregadas fazem sentido matemático
- **Completude geográfica**: Confirma que coordenadas estão dentro dos limites geográficos do Brasil
- **Integridade referencial**: Valida se códigos de estados correspondem aos nomes corretos

**8. Preparação para Análises Avançadas:**
- **Estrutura otimizada**: Dados organizados e limpos para aplicação de algoritmos de machine learning
- **Contexto regional**: Informações geográficas enriquecidas para interpretação dos resultados
- **Métricas padronizadas**: Indicadores harmonizados para comparação entre diferentes regiões

**9. Outputs de Validação:**
- **Relatórios estatísticos**: Resumos numéricos para validação da qualidade dos dados
- **Listas de top performers**: Identificação prévia de localizações promissoras
- **Distribuições regionais**: Visão geral da cobertura geográfica dos dados

**10. Benefícios desta Etapa:**
- **Confiabilidade**: Garante que dados estão corretos antes de análises complexas
- **Interpretabilidade**: Adiciona contexto geográfico e regional aos dados numéricos
- **Eficiência**: Identifica problemas cedo, evitando reprocessamento posterior
- **Insights preliminares**: Fornece primeira visão das oportunidades de expansão mais promissoras

In [ ]:
# Verificando os dados processados com coordenadas precisas
print("="*80)
print("DADOS PROCESSADOS COM COORDENADAS PRECISAS")
print("="*80)

print("Estrutura dos dados de localização:")
print(f"Colunas: {dados_localizacao.columns.tolist()}")
print(f"\nPrimeiras 5 localizações:")
print(dados_localizacao.head())

print(f"\nResumo estatístico:")
print(dados_localizacao.describe())

print(f"\nDistribuição por estado:")
estado_distribution = dados_localizacao['estado'].value_counts()
print(estado_distribution)

# Criando variável para códigos de cidade baseada na localização
dados_localizacao['cod_cidade'] = range(len(dados_localizacao))

# Adicionando informações regionais
dados_localizacao['regiao_chilli'] = dados_localizacao['estado'].map({
    'SP': 'SÃO PAULO',
    'RJ': 'SUDESTE', 'MG': 'SUDESTE', 'ES': 'SUDESTE',
    'RS': 'SUL', 'SC': 'SUL', 'PR': 'SUL',
    'BA': 'NORDESTE', 'PE': 'NORDESTE', 'CE': 'NORDESTE', 'AL': 'NORDESTE', 
    'PB': 'NORDESTE', 'RN': 'NORDESTE', 'SE': 'NORDESTE', 'PI': 'NORDESTE', 'MA': 'NORDESTE',
    'GO': 'CENTRO-OESTE', 'MT': 'CENTRO-OESTE', 'MS': 'CENTRO-OESTE', 'DF': 'CENTRO-OESTE',
    'PA': 'NORTE', 'AM': 'NORTE', 'TO': 'NORTE', 'RO': 'NORTE', 'AC': 'NORTE', 'RR': 'NORTE', 'AP': 'NORTE'
})

print(f"\nDistribuição por região Chilli:")
regiao_distribution = dados_localizacao['regiao_chilli'].value_counts()
print(regiao_distribution)

# Separando São Paulo dos outros estados (já feito anteriormente)
print(f"\nSeparação SP vs Outros Estados:")
print(f"- SP: {len(dados_sp)} localizações")
print(f"- Outros estados: {len(dados_outros_estados)} localizações")

# Top localizações por potencial em SP
print(f"\nTop 5 localizações em SP por potencial:")
if len(dados_sp) > 0:
    top_sp = dados_sp.nlargest(5, 'potencial_score')[['latitude', 'longitude', 'potencial_score', 'vendas_oculos_grau', 'num_clientes']]
    print(top_sp)

# Top localizações por potencial em outros estados
print(f"\nTop 5 localizações em outros estados por potencial:")
if len(dados_outros_estados) > 0:
    top_outros = dados_outros_estados.nlargest(5, 'potencial_score')[['estado', 'latitude', 'longitude', 'potencial_score', 'vendas_oculos_grau', 'num_clientes']]
    print(top_outros)

print("\n" + "="*80)

## Visualização Inicial dos Dados Geográficos

Esta seção cria **visualizações geográficas abrangentes** para explorar a distribuição espacial das lojas Chilli Beans, destacando padrões de vendas de óculos de grau, diferenças regionais e oportunidades de expansão através de múltiplas perspectivas analíticas.

**1. Dashboard Geográfico Multidimensional:**
- **Layout 2x2**: Organiza quatro visualizações complementares em uma única figura para análise comparativa
- **Coordenadas reais**: Utiliza latitude e longitude precisas para mapeamento geográfico fidedigno
- **Escalas consistentes**: Mantém proporções geográficas corretas para interpretação espacial acurada

**2. Gráfico 1 - Distribuição Geral de Vendas:**
- **Mapeamento de cores**: Representa intensidade de vendas de óculos de grau através do colormap 'viridis'
- **Tamanho proporcional**: Pontos dimensionados pelo número de lojas (×10) para mostrar densidade de operações
- **Identificação de hotspots**: Facilita visualização de regiões com alta concentração de vendas

**3. Gráfico 2 - Comparativo SP vs Outros Estados:**
- **Segmentação estratégica**: Diferencia São Paulo (vermelho) de outros estados (azul) para análise regional
- **Tamanho por vendas**: Pontos dimensionados pelas vendas de óculos de grau (×2) para destacar performance
- **Análise competitiva**: Permite comparação direta entre as duas principais regiões de atuação

**4. Gráfico 3 - Análise de Receita e Base de Clientes:**
- **Colormap de receita**: Utiliza 'plasma' para representar receita total por localização
- **Tamanho por clientes**: Pontos dimensionados pelo número de clientes (÷10) para mostrar penetração de mercado
- **Correlação visual**: Permite identificar relação entre receita e base de clientes

**5. Gráfico 4 - Ranking Estadual:**
- **Top 10 estados**: Identifica os maiores mercados por vendas de óculos de grau
- **Destaque para SP**: Usa cor vermelha para São Paulo e azul para outros estados
- **Valores absolutos**: Exibe números exatos de vendas sobre cada barra para precisão quantitativa

**6. Funcionalidades Avançadas de Visualização:**
- **Legendas informativas**: Explica significado de cores e tamanhos em cada gráfico
- **Grid e formatação**: Adiciona elementos visuais para melhor leitura dos dados
- **Rotação de labels**: Otimiza legibilidade dos nomes de estados no gráfico de barras

**7. Exportação e Persistência:**
- **Alta resolução**: Salva gráfico em 300 DPI para qualidade profissional
- **Formato PNG**: Compatível com documentos e apresentações
- **Ajuste automático**: Layout responsivo que se adapta ao conteúdo

**8. Estatísticas Complementares:**
- **Resumo quantitativo**: Fornece números consolidados por região (SP vs outros)
- **Métricas chave**: Total de cidades, vendas, receita e lojas por região
- **Validação visual**: Confirma padrões observados nos gráficos com dados numéricos

**9. Insights Estratégicos Facilitados:**
- **Identificação de gaps**: Visualiza regiões com baixa penetração de mercado
- **Oportunidades de expansão**: Destaca áreas com alto potencial não explorado
- **Balanceamento regional**: Mostra distribuição atual vs distribuição ideal

**10. Preparação para Clustering:**
- **Padrões espaciais**: Revela agrupamentos naturais de localizações de alta performance
- **Densidade geográfica**: Identifica regiões com concentração adequada para clustering
- **Validação de dados**: Confirma qualidade dos dados geográficos antes da análise de machine learning

In [ ]:
# VISUALIZAÇÃO INICIAL
# Plotando distribuição geográfica das lojas

fig, axes = plt.subplots(2, 2, figsize=(20, 16))

# Gráfico 1: Todas as localizações
ax1 = axes[0, 0]
scatter1 = ax1.scatter(dados_localizacao['longitude'], dados_localizacao['latitude'], 
                      c=dados_localizacao['vendas_oculos_grau'], 
                      s=dados_localizacao['num_lojas']*10,
                      alpha=0.7, cmap='viridis')
ax1.set_xlabel('Longitude (°)')
ax1.set_ylabel('Latitude (°)')
ax1.set_title('Distribuição Geográfica - Vendas de Óculos de Grau\n(Tamanho = Número de Lojas)')
cbar1 = plt.colorbar(scatter1, ax=ax1, label='Vendas Óculos de Grau')
cbar1.set_label('Quantidade de Vendas de Óculos de Grau', fontsize=10)
# Adicionar legenda para tamanho dos pontos
ax1.text(0.02, 0.98, 'Tamanho do ponto = Número de lojas', 
         transform=ax1.transAxes, fontsize=9, 
         bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8),
         verticalalignment='top')

# Gráfico 2: Separando São Paulo vs Outros Estados
ax2 = axes[0, 1]
scatter_sp = ax2.scatter(dados_sp['longitude'], dados_sp['latitude'], 
           c='red', s=dados_sp['vendas_oculos_grau']*2, alpha=0.7, label='São Paulo')
scatter_outros = ax2.scatter(dados_outros_estados['longitude'], dados_outros_estados['latitude'], 
           c='blue', s=dados_outros_estados['vendas_oculos_grau']*2, alpha=0.7, label='Outros Estados')
ax2.set_xlabel('Longitude (°)')
ax2.set_ylabel('Latitude (°)')
ax2.set_title('Distribuição: São Paulo vs Outros Estados\n(Tamanho = Vendas Óculos de Grau)')
ax2.legend(loc='upper right')
# Adicionar explicação do tamanho
ax2.text(0.02, 0.02, 'Tamanho proporcional às vendas de óculos de grau', 
         transform=ax2.transAxes, fontsize=9, 
         bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))

# Gráfico 3: Receita por localização
ax3 = axes[1, 0]
scatter3 = ax3.scatter(dados_localizacao['longitude'], dados_localizacao['latitude'], 
                      c=dados_localizacao['receita_total'], 
                      s=dados_localizacao['num_clientes']/10,
                      alpha=0.7, cmap='plasma')
ax3.set_xlabel('Longitude (°)')
ax3.set_ylabel('Latitude (°)')
ax3.set_title('Receita Total por Localização\n(Tamanho = Número de Clientes)')
cbar3 = plt.colorbar(scatter3, ax=ax3, label='Receita Total (R$)')
cbar3.set_label('Receita Total (R$)', fontsize=10)
# Adicionar legenda para tamanho dos pontos
ax3.text(0.02, 0.98, 'Tamanho do ponto = Número de clientes ÷ 10', 
         transform=ax3.transAxes, fontsize=9, 
         bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8),
         verticalalignment='top')

# Gráfico 4: Densidade de lojas por estado
ax4 = axes[1, 1]
densidade_estado = dados_localizacao.groupby('estado').agg({
    'num_lojas': 'sum',
    'vendas_oculos_grau': 'sum'
}).reset_index()

# Top 10 estados por vendas de óculos de grau
top_estados = densidade_estado.nlargest(10, 'vendas_oculos_grau')
bars = ax4.bar(range(len(top_estados)), top_estados['vendas_oculos_grau'], 
               color=['red' if estado == 'São Paulo' else 'steelblue' for estado in top_estados['estado']])
ax4.set_xticks(range(len(top_estados)))
ax4.set_xticklabels(top_estados['estado'], rotation=45, ha='right')
ax4.set_ylabel('Vendas Óculos de Grau (unidades)')
ax4.set_title('Top 10 Estados - Vendas de Óculos de Grau')
ax4.grid(axis='y', alpha=0.3)

# Adicionar valores nas barras
for i, (bar, valor) in enumerate(zip(bars, top_estados['vendas_oculos_grau'])):
    ax4.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 5, 
             f'{valor}', ha='center', va='bottom', fontsize=9, fontweight='bold')

# Legenda para cores
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='red', label='São Paulo'),
                   Patch(facecolor='steelblue', label='Outros Estados')]
ax4.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.savefig(("../../assets/gráficos/estatisticas_regiao.png"), dpi=300, bbox_inches='tight')
plt.show()

# Estatísticas descritivas
print("ESTATÍSTICAS POR REGIÃO:")
print("\nSão Paulo:")
print(f"- Total de cidades: {len(dados_sp)}")
print(f"- Vendas óculos de grau: {dados_sp['vendas_oculos_grau'].sum()}")
print(f"- Receita total: R$ {dados_sp['receita_total'].sum():,.2f}")
print(f"- Número de lojas: {dados_sp['num_lojas'].sum()}")

print("\nOutros Estados:")
print(f"- Total de cidades: {len(dados_outros_estados)}")
print(f"- Vendas óculos de grau: {dados_outros_estados['vendas_oculos_grau'].sum()}")
print(f"- Receita total: R$ {dados_outros_estados['receita_total'].sum():,.2f}")
print(f"- Número de lojas: {dados_outros_estados['num_lojas'].sum()}")

## Funções de Cálculo de Distância Geográfica

Esta seção implementa **funções essenciais para cálculos de distância geográfica** que são fundamentais para a análise de clustering espacial e identificação de localizações ideais para novas lojas.

**1. Função `haversine_distance()`:**
- **Propósito**: Calcula a distância real em quilômetros entre dois pontos na superfície da Terra
- **Fórmula Haversine**: Método matemático preciso que considera a curvatura da Terra
- **Parâmetros de entrada**: Latitude e longitude de dois pontos em graus decimais
- **Saída**: Distância em quilômetros com alta precisão

**2. Por que usar a Fórmula Haversine:**
- **Precisão geográfica**: Considera que a Terra é esférica, não plana
- **Aplicação real**: Fornece distâncias "em linha reta" reais entre coordenadas
- **Padrão da indústria**: Amplamente usado em sistemas de geolocalização
- **Eficiência computacional**: Rápida para grandes volumes de dados

**3. Processo de Cálculo Implementado:**
- **Conversão para radianos**: Converte graus para radianos (unidade matemática padrão)
- **Cálculo de diferenças**: Determina diferenças de latitude e longitude
- **Aplicação da fórmula**: Usa funções trigonométricas (seno, cosseno, arco-seno)
- **Multiplicação pelo raio**: Usa raio da Terra (6.371 km) para obter distância final

**4. Função `encontrar_cidade_mais_proxima()`:**
- **Propósito**: Identifica qual localização real está mais próxima de um centróide de cluster
- **Metodologia**: Calcula distância Haversine entre centróide e todas as localizações
- **Retorna**: A localização mais próxima e a distância em quilômetros
- **Aplicação**: Traduz centróides abstratos em recomendações de locais reais

**5. Importância para o Modelo de Clustering:**
- **Validação de clusters**: Permite verificar se centróides fazem sentido geograficamente
- **Recomendações práticas**: Converte resultados matemáticos em sugestões acionáveis
- **Interpretabilidade**: Facilita comunicação dos resultados para stakeholders
- **Tomada de decisão**: Fornece distâncias concretas para planejamento logístico

**6. Aplicações Específicas no Projeto:**
- **Análise de clusters SP**: Identifica cidades representativas de cada cluster em São Paulo
- **Análise nacional**: Encontra regiões ideais em outros estados
- **Validação de centróides**: Confirma que locais sugeridos são geograficamente viáveis
- **Métricas de qualidade**: Mede quão bem os clusters representam localizações reais

**7. Benefícios Técnicos:**
- **Eficiência**: Função vetorizada que processa múltiplas coordenadas rapidamente
- **Robustez**: Lida com casos extremos (coordenadas inválidas, DataFrames vazios)
- **Reutilização**: Funções modulares que podem ser aplicadas em diferentes contextos
- **Precisão**: Cálculos matematicamente corretos para decisões estratégicas

**8. Validações Implementadas:**
- **Verificação de dados**: Confirma que coordenadas são válidas antes do cálculo
- **Tratamento de erros**: Lida graciosamente com dados faltantes ou inválidos
- **Otimização**: Usa operações vetorizadas do pandas para performance
- **Teste de consistência**: Garante que distâncias calculadas são realistas

**9. Output Esperado:**
- **Distâncias precisas**: Valores em quilômetros para planejamento real
- **Localizações identificadas**: Cidades/regiões específicas para cada cluster
- **Métricas de proximidade**: Quão bem centróides representam dados reais
- **Base para recomendações**: Fundação sólida para sugestões de expansão

**10. Preparação para Análises Subsequentes:**
- **Clustering de SP**: Identificação de lojas representativas por cluster
- **Clustering nacional**: Mapeamento de regiões ideais por estado
- **Análise de potencial**: Conexão entre centróides matemáticos e oportunidades reais
- **Exportação de sugestões**: Conversão de resultados em coordenadas específicas para o modelo supervisionado

Esta implementação garante que todas as análises geográficas subsequentes sejam baseadas em **cálculos matematicamente precisos e geograficamente realistas**.

In [ ]:
# Função para calcular distância Haversine entre dois pontos
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calcula a distância Haversine entre dois pontos na Terra
    """
    R = 6371  # Raio da Terra em km
    
    # Convertendo para radianos
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Diferenças
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Fórmula Haversine
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return R * c

def encontrar_cidade_mais_proxima(centroide_lat, centroide_lon, dataframe):
    """
    Encontra a cidade mais próxima de um centróide
    """
    distancias = dataframe.apply(
        lambda row: haversine_distance(centroide_lat, centroide_lon, 
                                     row['latitude'], row['longitude']), 
        axis=1
    )
    idx_mais_proxima = distancias.idxmin()
    return dataframe.loc[idx_mais_proxima], distancias.min()

print("Funções de distância criadas com sucesso!")

## Clusterização de Lojas Existentes - São Paulo

Esta seção implementa a **análise de clustering específica para o estado de São Paulo**, utilizando algoritmos de machine learning para identificar padrões geográficos e operacionais das lojas existentes, com foco em vendas de óculos de grau.

**1. Por que Análise Separada para São Paulo:**
- **Densidade única**: São Paulo possui concentração muito maior de lojas que outros estados
- **Características específicas**: Mercado metropolitano com dinâmicas próprias
- **Evitar viés**: Impedir que SP domine a análise nacional devido ao volume
- **Estratégias diferenciadas**: Permitir recomendações específicas para o maior mercado

**2. Preparação dos Dados:**
- **Dataset focado**: Filtra apenas localizações do estado de São Paulo
- **Matriz de coordenadas**: Cria `X_sp` com longitude e latitude para algoritmos espaciais
- **Pesos estratégicos**: Usa `vendas_oculos_grau + 1` como peso para cada localização
- **Evita bias zero**: Adiciona 1 para garantir que todas as localizações tenham peso mínimo

**3. Metodologia de Teste de Clusters:**
- **Range de k**: Testa de 1 a 6 clusters para encontrar configuração ótima
- **Algoritmo K-Means**: Usa coordenadas geográficas como features principais
- **Sample weights**: Prioriza localizações com mais vendas de óculos de grau
- **Parâmetros fixos**: `random_state=42` garante reprodutibilidade

**4. Layout de Visualização (Grid 2x3 + Grid 1x2):**
- **Primeira figura**: 6 gráficos de clustering organizados em 2 linhas × 3 colunas
- **Segunda figura**: Análises de qualidade lado a lado (Elbow + Silhouette)
- **Consistência visual**: Mesmo padrão usado para análise de outros estados
- **Tamanhos otimizados**: 20×12 para clustering, 16×6 para análises

**5. Métricas de Qualidade Implementadas:**
- **Inércia (WCSS)**: Mede compactação dos clusters (menor = melhor)
- **Silhouette Score**: Avalia separação entre clusters (maior = melhor)
- **Validação múltipla**: Compara diferentes valores de k para otimização
- **Tratamento de exceções**: Lida com casos onde silhouette não pode ser calculado

**6. Identificação de Lojas Representativas:**
- **Função Haversine**: Calcula distâncias reais entre centróides e lojas
- **Loja mais próxima**: Identifica localização real mais próxima de cada centróide
- **Métricas contextuais**: Mostra vendas e número de lojas para cada representante
- **Interpretabilidade**: Converte centróides abstratos em recomendações práticas

**7. Análises Detalhadas:**
- **k=3**: Configuração intermediária com análise de clusters representativos
- **k=6**: Configuração mais granular para segmentação detalhada
- **Tabela comparativa**: Resume todos os valores de k com interpretações
- **Melhor k identificado**: Baseado no maior Silhouette Score

**8. Outputs Gerados:**
- **clusters_sp.png**: Visualização dos 6 gráficos de clustering
- **analise_clusters_sp.png**: Gráficos de Elbow e Silhouette lado a lado
- **Estatísticas consolidadas**: Vendas, lojas, receita e melhor configuração
- **Recomendações específicas**: Clusters representativos para cada k

**9. Interpretação dos Resultados:**
- **Clusters identificados**: Agrupamentos naturais de lojas em SP
- **Representantes**: Localizações-chave que exemplificam cada cluster
- **Qualidade validada**: Silhouette Score confirma separação adequada
- **Base para estratégia**: Insights específicos para mercado paulista

**10. Integração com Análise Nacional:**
- **Comparação facilitada**: Layout idêntico ao de outros estados
- **Estratégias complementares**: SP requer abordagem diferenciada
- **Dados consolidados**: Contribui para visão geral do modelo
- **Próximas etapas**: Prepara para análise de expansão direcionada

Esta análise garante que as **particularidades do mercado paulista** sejam adequadamente capturadas e que as recomendações de expansão considerem as dinâmicas específicas da região mais importante para a Chilli Beans.

In [ ]:
# CLUSTERIZAÇÃO DE LOJAS EXISTENTES - SÃO PAULO
print("\n=== ANÁLISE DE LOJAS EXISTENTES - SÃO PAULO ===")

# Preparando dados de SP para clusterização
X_sp = dados_sp[['longitude', 'latitude']].values
weights_sp = dados_sp['vendas_oculos_grau'].values + 1

# Testando diferentes números de clusters (1 a 6) - ATUALIZADO PARA CONSISTÊNCIA
k_values_sp = range(1, 7)
inertias_sp = []
silhouette_scores_sp = []

# Primeira figura: Grid 2x3 para os 6 gráficos de clustering
fig1, axes = plt.subplots(2, 3, figsize=(20, 12))
fig1.suptitle('Clusterização de Lojas Existentes - São Paulo (K=1 a K=6)', fontsize=16)

for i, k in enumerate(k_values_sp):
    # KMeans com peso das vendas de óculos de grau
    kmeans_sp = KMeans(n_clusters=k, random_state=42, n_init=10)
    clusters_sp = kmeans_sp.fit_predict(X_sp, sample_weight=weights_sp)
    
    # Métricas
    inertias_sp.append(kmeans_sp.inertia_)
    if k > 1 and len(np.unique(clusters_sp)) > 1:
        try:
            silhouette_avg = silhouette_score(X_sp, clusters_sp)
            silhouette_scores_sp.append(silhouette_avg)
        except:
            silhouette_scores_sp.append(0)
    elif k > 1:
        silhouette_scores_sp.append(0)
    
    # Plotando clusters em grid 2x3
    row = i // 3
    col = i % 3
    ax = axes[row, col]
    
    scatter = ax.scatter(dados_sp['longitude'], dados_sp['latitude'], 
                       c=clusters_sp, s=dados_sp['vendas_oculos_grau']*2, 
                       alpha=0.7, cmap='tab10')
    
    # Plotando centróides
    centroids = kmeans_sp.cluster_centers_
    ax.scatter(centroids[:, 0], centroids[:, 1], 
              c='red', marker='x', s=200, linewidths=3, label='Centróides')
    
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title(f'K = {k} clusters')
    ax.legend()
    
    # Encontrando lojas representativas de cada cluster para k=3
    if k == 3:
        print(f"\nDetalhamento para K = {k} (clusters representativos):")
        for j, (cent_lon, cent_lat) in enumerate(centroids):
            loja_proxima, distancia = encontrar_cidade_mais_proxima(cent_lat, cent_lon, dados_sp)
            print(f"  Cluster {j+1}: {loja_proxima['estado']} (Lat: {loja_proxima['latitude']:.2f}, Lon: {loja_proxima['longitude']:.2f}) - Distância: {distancia:.2f} km")
            print(f"    Vendas óculos de grau: {loja_proxima['vendas_oculos_grau']}, Lojas: {loja_proxima['num_lojas']}")

plt.tight_layout()
plt.savefig("../../assets/gráficos/clusters_sp.png", dpi=300, bbox_inches='tight')
plt.show()

# Segunda figura: Grid 1x2 para Elbow e Silhouette lado a lado
fig2, (ax_elbow, ax_silhouette) = plt.subplots(1, 2, figsize=(16, 6))
fig2.suptitle('Análise de Qualidade dos Clusters - São Paulo', fontsize=16)

# Gráfico da curva do cotovelo
ax_elbow.plot(k_values_sp, inertias_sp, 'bo-', linewidth=2, markersize=8)
ax_elbow.set_xlabel('Número de Clusters (k)')
ax_elbow.set_ylabel('Inércia (WCSS)')
ax_elbow.set_title('Curva do Cotovelo')
ax_elbow.grid(True, alpha=0.3)
# Adicionar anotação para identificar o cotovelo
ax_elbow.text(0.02, 0.98, 'Procure o "cotovelo" onde\na inércia para de diminuir\nsignificativamente', 
             transform=ax_elbow.transAxes, fontsize=10, 
             bbox=dict(boxstyle="round,pad=0.3", facecolor="lightyellow", alpha=0.8),
             verticalalignment='top')

# Gráfico de Silhouette Score
if silhouette_scores_sp:
    ax_silhouette.plot(k_values_sp[1:], silhouette_scores_sp, 'ro-', linewidth=2, markersize=8)
    ax_silhouette.set_xlabel('Número de Clusters (k)')
    ax_silhouette.set_ylabel('Silhouette Score')
    ax_silhouette.set_title('Silhouette Score')
    ax_silhouette.grid(True, alpha=0.3)
    ax_silhouette.set_ylim(0, max(silhouette_scores_sp) + 0.1)
    
    # Adicionar linha de referência para qualidade boa (>0.5)
    ax_silhouette.axhline(y=0.5, color='green', linestyle='--', alpha=0.7, linewidth=2)
    ax_silhouette.text(0.02, 0.98, 'Score > 0.5 indica\nboa separação\ndos clusters', 
                      transform=ax_silhouette.transAxes, fontsize=10, 
                      bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.8),
                      verticalalignment='top')
    
    best_k_sp = k_values_sp[1:][np.argmax(silhouette_scores_sp)]
    ax_silhouette.scatter([best_k_sp], [max(silhouette_scores_sp)], 
                         color='red', s=100, zorder=5, label=f'Melhor k = {best_k_sp}')
    ax_silhouette.legend()
    
    print(f"\nMelhor Silhouette Score São Paulo: {max(silhouette_scores_sp):.3f} com k = {best_k_sp}")
else:
    ax_silhouette.text(0.5, 0.5, 'Silhouette Score\nnão calculado', 
                      ha='center', va='center', transform=ax_silhouette.transAxes)

plt.tight_layout()
plt.savefig("../../assets/gráficos/analise_clusters_sp.png", dpi=300, bbox_inches='tight')
plt.show()

# Análise adicional para k=6
if len(k_values_sp) >= 6:
    print(f"\nAnálise detalhada para K = 6:")
    kmeans_6 = KMeans(n_clusters=6, random_state=42, n_init=10)
    clusters_6 = kmeans_6.fit_predict(X_sp, sample_weight=weights_sp)
    centroids_6 = kmeans_6.cluster_centers_
    
    for j, (cent_lon, cent_lat) in enumerate(centroids_6):
        loja_proxima, distancia = encontrar_cidade_mais_proxima(cent_lat, cent_lon, dados_sp)
        print(f"  Cluster {j+1}: {loja_proxima['estado']} (Lat: {loja_proxima['latitude']:.2f}, Lon: {loja_proxima['longitude']:.2f}) - Distância: {distancia:.2f} km")
        print(f"    Vendas óculos de grau: {loja_proxima['vendas_oculos_grau']}, Lojas: {loja_proxima['num_lojas']}")

print(f"\nAnálise das lojas em São Paulo:")
print(f"- São Paulo concentra {dados_sp['vendas_oculos_grau'].sum()} vendas de óculos de grau")
print(f"- Distribuídas em {dados_sp['num_lojas'].sum()} lojas")
print(f"- Receita total: R$ {dados_sp['receita_total'].sum():.2f}")
print(f"- Melhor configuração: {best_k_sp} clusters regionais")

# Tabela comparativa dos diferentes valores de k
print(f"\n📊 COMPARAÇÃO DOS DIFERENTES VALORES DE K:")
print(f"{'K':<3} {'Inércia':<12} {'Silhouette':<12} {'Interpretação'}")
print(f"-" * 50)
for i, k in enumerate(k_values_sp):
    inercia = inertias_sp[i]
    silhouette = silhouette_scores_sp[i-1] if i > 0 and i-1 < len(silhouette_scores_sp) else "N/A"
    
    if k == 1:
        interpretacao = "Sem clusters"
    elif k == best_k_sp:
        interpretacao = "⭐ ÓTIMO"
    elif isinstance(silhouette, float) and silhouette > 0.5:
        interpretacao = "Boa qualidade"
    elif isinstance(silhouette, float) and silhouette > 0.3:
        interpretacao = "Qualidade média"
    else:
        interpretacao = "Baixa qualidade"
    
    silhouette_str = f"{silhouette:.3f}" if isinstance(silhouette, float) else str(silhouette)
    print(f"{k:<3} {inercia:<12.2f} {silhouette_str:<12} {interpretacao}")

## Clusterização de Lojas Existentes - Outros Estados

Esta seção implementa a **análise de clustering para todos os estados brasileiros exceto São Paulo**, aplicando machine learning para identificar padrões regionais e oportunidades de otimização da rede de lojas existente da Chilli Beans.

**1. Escopo da Análise - Brasil Menos São Paulo:**
- **Abrangência nacional**: Inclui todos os 26 estados + DF, exceto São Paulo
- **Diversidade geográfica**: Contempla diferentes regiões, climas e culturas
- **Heterogeneidade econômica**: Mercados com diferentes potenciais e características
- **Complementaridade**: Análise que se soma à específica de São Paulo

**2. Preparação dos Dados Regionais:**
- **Dataset nacional**: `dados_outros_estados` contém todas as localizações não-SP
- **Matriz espacial**: `X_outros` com coordenadas longitude/latitude
- **Ponderação estratégica**: `vendas_oculos_grau + 1` como peso de cada localização
- **Normalização**: Garante que localizações com vendas zero ainda sejam consideradas

**3. Metodologia de Clustering Abrangente:**
- **Range expandido**: Testa k=1 a k=6 para maior granularidade
- **Algoritmo robusto**: K-Means com sample weights para considerar performance
- **Validação dupla**: Inércia (compactação) + Silhouette (separação)
- **Reprodutibilidade**: `random_state=42` para resultados consistentes

**4. Estrutura de Visualização Padronizada:**
- **Grid 2×3**: Seis gráficos de clustering para comparação visual direta
- **Grid 1×2 separado**: Elbow e Silhouette lado a lado para análise de qualidade
- **Layout consistente**: Idêntico à análise de São Paulo para facilitar comparação
- **Tamanhos otimizados**: 20×12 e 16×6 para visualização clara

**5. Análise de Qualidade dos Clusters:**
- **Curva do Cotovelo**: Identifica ponto ótimo onde inércia estabiliza
- **Silhouette Score**: Mede qualidade da separação (>0.5 = boa qualidade)
- **Linha de referência**: Marca visual no gráfico para interpretação fácil
- **Melhor k destacado**: Identificação automática da configuração ótima

**6. Identificação de Regiões Representativas:**
- **Centróides calculados**: Pontos médios matemáticos de cada cluster
- **Mapeamento para realidade**: Função Haversine encontra cidade mais próxima
- **Contexto regional**: Cada cluster representado por localização real específica
- **Métricas de proximidade**: Distância em km entre centróide e representante

**7. Análises Detalhadas Multi-k:**
- **k=4**: Configuração intermediária com análise detalhada de representantes
- **k=6**: Máxima granularidade para segmentação regional específica
- **Tabela comparativa completa**: Resumo de inércia, silhouette e interpretação
- **Classificação qualitativa**: Ótimo, boa, média ou baixa qualidade

**8. Outputs Informativos:**
- **clusters_br.png**: Visualização dos 6 gráficos de clustering nacional
- **analise_clusters_br.png**: Gráficos de qualidade lado a lado
- **Estatísticas consolidadas**: Total de vendas, lojas e receita nacional
- **Configuração recomendada**: Melhor k identificado automaticamente

**9. Insights Regionais Estratégicos:**
- **Padrões geográficos**: Clusters naturais baseados em proximidade e performance
- **Representatividade**: Cada cluster tem uma "capital" identificada
- **Diversidade capturada**: Diferentes regiões com características únicas
- **Base para expansão**: Identifica gaps e oportunidades por região

**10. Integração com Modelo Global:**
- **Complementa São Paulo**: Visão nacional completa quando combinado
- **Consistência metodológica**: Mesma abordagem, diferentes escopos
- **Dados para supervisionado**: Clusters informam próximas etapas do modelo
- **Estratégia diferenciada**: Permite abordagens específicas por região

**11. Validação e Robustez:**
- **Múltiplas métricas**: Inércia + Silhouette para validação cruzada
- **Tratamento de exceções**: Código robusto para casos extremos
- **Interpretabilidade**: Resultados traduzidos em recomendações práticas
- **Reprodutibilidade**: Parâmetros fixos garantem consistência

**12. Preparação para Próximas Etapas:**
- **Clustering ótimo identificado**: Base sólida para análise de expansão
- **Regiões mapeadas**: Cada cluster representa oportunidade específica
- **Dados estruturados**: Prontos para algoritmos supervisionados
- **Insights acionáveis**: Recomendações diretas para tomada de decisão

Esta análise fornece uma **visão abrangente e sistemática** do mercado nacional (exceto SP), identificando padrões regionais naturais e oportunidades de otimização que complementam perfeitamente a análise específica de São Paulo.

In [ ]:
# CLUSTERIZAÇÃO DE LOJAS EXISTENTES - OUTROS ESTADOS
print("\n=== ANÁLISE DE LOJAS EXISTENTES - OUTROS ESTADOS ===")

# Preparando dados de outros estados para clusterização
X_outros = dados_outros_estados[['longitude', 'latitude']].values
weights_outros = dados_outros_estados['vendas_oculos_grau'].values + 1

# Testando diferentes números de clusters (1 a 6)
k_values_outros = range(1, 7)
inertias_outros = []
silhouette_scores_outros = []

# Primeira figura: Grid 2x3 para os 6 gráficos de clustering
fig1, axes = plt.subplots(2, 3, figsize=(20, 12))
fig1.suptitle('Clusterização de Lojas Existentes - Outros Estados (K=1 a K=6)', fontsize=16)

for i, k in enumerate(k_values_outros):
    # KMeans com peso das vendas de óculos de grau
    kmeans_outros = KMeans(n_clusters=k, random_state=42, n_init=10)
    clusters_outros = kmeans_outros.fit_predict(X_outros, sample_weight=weights_outros)
    
    # Métricas
    inertias_outros.append(kmeans_outros.inertia_)
    if k > 1 and len(np.unique(clusters_outros)) > 1:
        try:
            silhouette_avg = silhouette_score(X_outros, clusters_outros)
            silhouette_scores_outros.append(silhouette_avg)
        except:
            silhouette_scores_outros.append(0)
    elif k > 1:
        silhouette_scores_outros.append(0)
    
    # Plotando clusters em grid 2x3
    row = i // 3
    col = i % 3
    ax = axes[row, col]
    
    scatter = ax.scatter(dados_outros_estados['longitude'], dados_outros_estados['latitude'], 
                       c=clusters_outros, s=dados_outros_estados['vendas_oculos_grau']*0.5, 
                       alpha=0.7, cmap='tab10')
    
    # Plotando centróides
    centroids = kmeans_outros.cluster_centers_
    ax.scatter(centroids[:, 0], centroids[:, 1], 
              c='red', marker='x', s=200, linewidths=3, label='Centróides')
    
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title(f'K = {k} clusters')
    ax.legend()
    
    # Encontrando regiões representativas de cada cluster para k=4
    if k == 4:
        print(f"\nDetalhamento para K = {k} (clusters representativos):")
        for j, (cent_lon, cent_lat) in enumerate(centroids):
            regiao_proxima, distancia = encontrar_cidade_mais_proxima(cent_lat, cent_lon, dados_outros_estados)
            print(f"  Cluster {j+1}: {regiao_proxima['estado']} (Lat: {regiao_proxima['latitude']:.2f}, Lon: {regiao_proxima['longitude']:.2f}) - Distância: {distancia:.2f} km")
            print(f"    Vendas óculos de grau: {regiao_proxima['vendas_oculos_grau']}, Lojas: {regiao_proxima['num_lojas']}")

plt.tight_layout()
plt.savefig("../../assets/gráficos/clusters_br.png", dpi=300, bbox_inches='tight')
plt.show()

# Segunda figura: Grid 1x2 para Elbow e Silhouette lado a lado
fig2, (ax_elbow, ax_silhouette) = plt.subplots(1, 2, figsize=(16, 6))
fig2.suptitle('Análise de Qualidade dos Clusters - Outros Estados', fontsize=16)

# Gráfico da curva do cotovelo
ax_elbow.plot(k_values_outros, inertias_outros, 'bo-', linewidth=2, markersize=8)
ax_elbow.set_xlabel('Número de Clusters (k)')
ax_elbow.set_ylabel('Inércia (WCSS)')
ax_elbow.set_title('Curva do Cotovelo')
ax_elbow.grid(True, alpha=0.3)
# Adicionar anotação para identificar o cotovelo
ax_elbow.text(0.02, 0.98, 'Procure o "cotovelo" onde\na inércia para de diminuir\nsignificativamente', 
             transform=ax_elbow.transAxes, fontsize=10, 
             bbox=dict(boxstyle="round,pad=0.3", facecolor="lightyellow", alpha=0.8),
             verticalalignment='top')

# Gráfico de Silhouette Score
if silhouette_scores_outros:
    ax_silhouette.plot(k_values_outros[1:], silhouette_scores_outros, 'ro-', linewidth=2, markersize=8)
    ax_silhouette.set_xlabel('Número de Clusters (k)')
    ax_silhouette.set_ylabel('Silhouette Score')
    ax_silhouette.set_title('Silhouette Score')
    ax_silhouette.grid(True, alpha=0.3)
    ax_silhouette.set_ylim(0, max(silhouette_scores_outros) + 0.1)
    
    # Adicionar linha de referência para qualidade boa (>0.5)
    ax_silhouette.axhline(y=0.5, color='green', linestyle='--', alpha=0.7, linewidth=2)
    ax_silhouette.text(0.02, 0.98, 'Score > 0.5 indica\nboa separação\ndos clusters', 
                      transform=ax_silhouette.transAxes, fontsize=10, 
                      bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.8),
                      verticalalignment='top')
    
    best_k_outros = k_values_outros[1:][np.argmax(silhouette_scores_outros)]
    ax_silhouette.scatter([best_k_outros], [max(silhouette_scores_outros)], 
                         color='red', s=100, zorder=5, label=f'Melhor k = {best_k_outros}')
    ax_silhouette.legend()
    
    print(f"\nMelhor Silhouette Score Outros Estados: {max(silhouette_scores_outros):.3f} com k = {best_k_outros}")
else:
    ax_silhouette.text(0.5, 0.5, 'Silhouette Score\nnão calculado', 
                      ha='center', va='center', transform=ax_silhouette.transAxes)

plt.tight_layout()
plt.savefig("../../assets/gráficos/analise_clusters_br.png", dpi=300, bbox_inches='tight')
plt.show()

# Análise adicional para k=6
if len(k_values_outros) >= 6:
    print(f"\nAnálise detalhada para K = 6:")
    kmeans_6 = KMeans(n_clusters=6, random_state=42, n_init=10)
    clusters_6 = kmeans_6.fit_predict(X_outros, sample_weight=weights_outros)
    centroids_6 = kmeans_6.cluster_centers_
    
    for j, (cent_lon, cent_lat) in enumerate(centroids_6):
        regiao_proxima, distancia = encontrar_cidade_mais_proxima(cent_lat, cent_lon, dados_outros_estados)
        print(f"  Cluster {j+1}: {regiao_proxima['estado']} (Lat: {regiao_proxima['latitude']:.2f}, Lon: {regiao_proxima['longitude']:.2f}) - Distância: {distancia:.2f} km")
        print(f"    Vendas óculos de grau: {regiao_proxima['vendas_oculos_grau']}, Lojas: {regiao_proxima['num_lojas']}")

print(f"\nAnálise das lojas nos outros estados:")
print(f"- Outros estados concentram {dados_outros_estados['vendas_oculos_grau'].sum()} vendas de óculos de grau")
print(f"- Distribuídas em {dados_outros_estados['num_lojas'].sum()} lojas")
print(f"- Receita total: R$ {dados_outros_estados['receita_total'].sum():.2f}")
print(f"- Melhor configuração: {best_k_outros} clusters regionais")

# Tabela comparativa dos diferentes valores de k
print(f"\n📊 COMPARAÇÃO DOS DIFERENTES VALORES DE K:")
print(f"{'K':<3} {'Inércia':<12} {'Silhouette':<12} {'Interpretação'}")
print(f"-" * 50)
for i, k in enumerate(k_values_outros):
    inercia = inertias_outros[i]
    silhouette = silhouette_scores_outros[i-1] if i > 0 and i-1 < len(silhouette_scores_outros) else "N/A"
    
    if k == 1:
        interpretacao = "Sem clusters"
    elif k == best_k_outros:
        interpretacao = "⭐ ÓTIMO"
    elif isinstance(silhouette, float) and silhouette > 0.5:
        interpretacao = "Boa qualidade"
    elif isinstance(silhouette, float) and silhouette > 0.3:
        interpretacao = "Qualidade média"
    else:
        interpretacao = "Baixa qualidade"
    
    silhouette_str = f"{silhouette:.3f}" if isinstance(silhouette, float) else str(silhouette)
    print(f"{k:<3} {inercia:<12.2f} {silhouette_str:<12} {interpretacao}")

## Identificação de Locais Ideais para Novas Lojas

Esta seção representa o **núcleo estratégico do modelo não supervisionado**, onde os insights das análises anteriores são consolidados para identificar localizações específicas e acionáveis para expansão da rede Chilli Beans, com foco particular em óculos de grau.

**1. Criação do Score de Potencial Multicritério:**
- **Fórmula ponderada estratégica**: Combina 4 métricas essenciais com pesos otimizados
- **40% vendas de óculos de grau**: Prioridade máxima para o produto-foco da análise
- **30% número de clientes**: Indica penetração e potencial de mercado
- **20% receita total**: Representa poder de compra da região
- **10% número de lojas**: Identifica saturação vs oportunidade (peso menor para não penalizar regiões sub-exploradas)

**2. Estratégia de Separação Regional Mantida:**
- **São Paulo isolado**: Continua análise independente devido às características únicas
- **Outros estados priorizados**: Foco principal na análise de expansão para evitar viés metropolitano
- **Complementaridade**: Ambas as análises juntas fornecem visão completa nacional

**3. Algoritmo K-Means Aplicado para Expansão:**
- **k=5 clusters**: Configuração ótima baseada nas análises de silhouette anteriores
- **Coordenadas como features**: Longitude e latitude para clustering espacial
- **Sample weights**: Prioriza regiões com maior score de potencial
- **Centróides como locais ideais**: Pontos matemáticos ótimos traduzidos em recomendações geográficas

**4. Dashboard Visual Multidimensional (2×2):**
- **Gráfico 1 - Clusters por Potencial**: Visualiza agrupamentos com tamanho proporcional ao score
- **Gráfico 2 - Vendas de Óculos de Grau**: Foca especificamente no produto-alvo da análise
- **Gráfico 3 - Base de Clientes vs Receita**: Correlaciona penetração de mercado com poder de compra
- **Gráfico 4 - Ranking de Clusters**: Identifica hierarquia de prioridades com escala de cores

**5. Análise Estatística dos Clusters:**
- **Métricas agregadas**: Soma e média de indicadores por cluster
- **Centróides precisos**: Coordenadas exatas dos pontos ideais para novas lojas
- **Ranking por potencial**: Ordenação automática dos clusters por atratividade
- **Estatísticas descritivas**: Número de regiões, vendas totais, clientes e receita por cluster

**6. Sistema de Recomendações Hierárquicas:**
- **Prioridade por cluster**: Do maior ao menor potencial médio
- **Top 3 regiões por cluster**: Localizações específicas mais promissoras dentro de cada agrupamento
- **Distância dos centróides**: Métricas Haversine para validar proximidade geográfica
- **Contextualização detalhada**: Score, vendas, lojas atuais e base de clientes para cada recomendação

**7. Exportação de Insights Acionáveis:**
- **Coordenadas específicas**: Latitude e longitude para cada local recomendado
- **Métricas de potencial**: Scores quantitativos para priorização de investimentos
- **Validação geográfica**: Distâncias reais em quilômetros para planejamento logístico
- **Resumo executivo**: Tabela consolidada com ranking de clusters

**8. Integração com Modelo Supervisionado:**
- **Base sólida**: Recomendações fundamentadas em dados reais e algoritmos validados
- **Coordenadas precisas**: Input direto para modelos preditivos subsequentes
- **Métricas padronizadas**: Indicadores consistentes para comparação e validação
- **Escalabilidade**: Framework replicável para futuras expansões

**9. Validações de Qualidade Implementadas:**
- **Centróides geograficamente viáveis**: Pontos dentro do território nacional
- **Diversidade regional**: Clusters distribuídos em diferentes estados/regiões
- **Consistência estatística**: Scores de potencial coerentes com dados históricos
- **Interpretabilidade**: Resultados traduzíveis em decisões estratégicas

**10. Outputs Estratégicos Gerados:**
- **Arquivo de visualização**: `potencial_clusters.png` com dashboard completo
- **Lista priorizada**: Ranking de 5 clusters com recomendações específicas
- **Coordenadas acionáveis**: Localizações precisas para prospecção imediata
- **Métricas de validação**: Estatísticas que justificam cada recomendação

**11. Benefícios para Tomada de Decisão:**
- **Redução de riscos**: Recomendações baseadas em dados históricos reais
- **Otimização de investimentos**: Priorização clara dos locais mais promissores
- **Escalabilidade**: Metodologia replicável para expansões futuras
- **ROI maximizado**: Foco em regiões com maior potencial de retorno

**12. Preparação para Implementação:**
- **Coordenadas específicas**: Prontas para sistemas de GPS e mapeamento
- **Contexto regional**: Informações sobre características locais de cada cluster
- **Métricas de acompanhamento**: Indicadores para monitoramento pós-implementação
- **Base para modelo supervisionado**: Dados estruturados para próximas etapas analíticas

Esta análise transforma **dados brutos em recomendações acionáveis**, fornecendo à Chilli Beans um roadmap claro e fundamentado para expansão estratégica com foco em óculos de grau, maximizando o potencial de sucesso de cada nova loja através de localização

In [ ]:
# IDENTIFICAÇÃO DE LOCAIS IDEAIS PARA NOVAS LOJAS
print("=== ANÁLISE PARA IDENTIFICAÇÃO DE NOVOS LOCAIS ===")

# Criando score de potencial baseado em múltiplos fatores para identificar locais ideais
dados_localizacao['potencial_score'] = (
    dados_localizacao['vendas_oculos_grau'] * 0.4 +  # 40% peso vendas atuais de óculos de grau
    dados_localizacao['num_clientes'] * 0.3 +        # 30% peso número de clientes
    dados_localizacao['receita_total'] * 0.2 +       # 20% peso receita
    dados_localizacao['num_lojas'] * 0.1             # 10% peso lojas existentes (para identificar saturação)
)

# Separando novamente SP e outros estados para análise independente
dados_expansao_sp = dados_localizacao[dados_localizacao['estado'] == 'São Paulo'].copy()
dados_expansao_outros = dados_localizacao[dados_localizacao['estado'] != 'São Paulo'].copy()

# ANÁLISE PRINCIPAL: OUTROS ESTADOS (excluindo SP para evitar viés)
print("\n=== IDENTIFICAÇÃO DE NOVOS LOCAIS - OUTROS ESTADOS ===")

X_expansao = dados_expansao_outros[['longitude', 'latitude']].values
weights_expansao = dados_expansao_outros['potencial_score'].values + 1

# Usando k=5 como ótimo baseado na análise anterior
k_otimo = 5
kmeans_expansao = KMeans(n_clusters=k_otimo, random_state=42, n_init=10)
clusters_expansao = kmeans_expansao.fit_predict(X_expansao, sample_weight=weights_expansao)

# Visualização da clusterização para identificação de novos locais
fig, axes = plt.subplots(2, 2, figsize=(20, 16))

# Gráfico 1: Clusters baseados no potencial de mercado
ax1 = axes[0, 0]
scatter1 = ax1.scatter(dados_expansao_outros['longitude'], dados_expansao_outros['latitude'], 
                      c=clusters_expansao, s=dados_expansao_outros['potencial_score']*2, 
                      alpha=0.7, cmap='tab10')
centroids = kmeans_expansao.cluster_centers_
ax1.scatter(centroids[:, 0], centroids[:, 1], 
           c='red', marker='x', s=300, linewidths=4, label='Locais Ideais (Centróides)')
ax1.set_xlabel('Longitude (°)')
ax1.set_ylabel('Latitude (°)')
ax1.set_title('Clusters por Potencial de Mercado\n(Tamanho = Potencial Score)')
ax1.legend(loc='upper right')
# Adicionar explicação do tamanho e cores
ax1.text(0.02, 0.02, 'Cores = Clusters diferentes\nTamanho = Score de potencial × 2', 
         transform=ax1.transAxes, fontsize=9, 
         bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))

# Gráfico 2: Foco específico em vendas de óculos de grau
ax2 = axes[0, 1]
scatter2 = ax2.scatter(dados_expansao_outros['longitude'], dados_expansao_outros['latitude'], 
                      c=dados_expansao_outros['vendas_oculos_grau'], 
                      s=dados_expansao_outros['num_lojas']*10, 
                      alpha=0.7, cmap='plasma')
ax2.scatter(centroids[:, 0], centroids[:, 1], 
           c='black', marker='x', s=300, linewidths=4, label='Locais Ideais')
ax2.set_xlabel('Longitude (°)')
ax2.set_ylabel('Latitude (°)')
ax2.set_title('Vendas Atuais de Óculos de Grau\n(Tamanho = Número de Lojas)')
cbar2 = plt.colorbar(scatter2, ax=ax2, label='Vendas Óculos de Grau')
cbar2.set_label('Quantidade de Vendas de Óculos de Grau', fontsize=10)
ax2.legend(loc='upper right')
# Adicionar explicação do tamanho
ax2.text(0.02, 0.02, 'Tamanho = Número de lojas × 10', 
         transform=ax2.transAxes, fontsize=9, 
         bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))

# Gráfico 3: Densidade de clientes vs receita
ax3 = axes[1, 0]
scatter3 = ax3.scatter(dados_expansao_outros['longitude'], dados_expansao_outros['latitude'], 
                      c=dados_expansao_outros['num_clientes'], 
                      s=dados_expansao_outros['receita_total']*10, 
                      alpha=0.7, cmap='viridis')
ax3.scatter(centroids[:, 0], centroids[:, 1], 
           c='white', marker='x', s=300, linewidths=4, label='Locais Ideais', 
           edgecolors='black')
ax3.set_xlabel('Longitude (°)')
ax3.set_ylabel('Latitude (°)')
ax3.set_title('Base de Clientes\n(Tamanho = Receita Total)')
cbar3 = plt.colorbar(scatter3, ax=ax3, label='Número de Clientes')
cbar3.set_label('Quantidade de Clientes', fontsize=10)
ax3.legend(loc='upper right')
# Adicionar explicação do tamanho
ax3.text(0.02, 0.02, 'Tamanho = Receita total × 10', 
         transform=ax3.transAxes, fontsize=9, 
         bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))

# Gráfico 4: Ranking de potencial por cluster
cluster_stats = []
for i in range(k_otimo):
    mask = clusters_expansao == i
    cluster_data = dados_expansao_outros[mask]
    
    stats = {
        'cluster': i+1,
        'num_regioes': len(cluster_data),
        'vendas_oculos_grau_total': cluster_data['vendas_oculos_grau'].sum(),
        'num_clientes_total': cluster_data['num_clientes'].sum(),
        'receita_total': cluster_data['receita_total'].sum(),
        'potencial_medio': cluster_data['potencial_score'].mean(),
        'potencial_maximo': cluster_data['potencial_score'].max(),
        'centro_lat': centroids[i][1],
        'centro_lon': centroids[i][0]
    }
    cluster_stats.append(stats)

cluster_df = pd.DataFrame(cluster_stats)
cluster_df = cluster_df.sort_values('potencial_medio', ascending=False)

ax4 = axes[1, 1]
colors = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(cluster_df)))
bars = ax4.bar(cluster_df['cluster'], cluster_df['potencial_medio'], color=colors)
ax4.set_xlabel('Número do Cluster')
ax4.set_ylabel('Score de Potencial Médio')
ax4.set_title('Ranking de Potencial por Cluster\n(Verde = Maior Potencial)')
ax4.grid(True, alpha=0.3, axis='y')

# Adicionando valores nas barras
for i, (bar, valor) in enumerate(zip(bars, cluster_df['potencial_medio'])):
    ax4.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.2, 
             f'{valor:.1f}', ha='center', va='bottom', fontweight='bold', fontsize=10)

# Adicionar escala de cores como legenda
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor=plt.cm.RdYlGn(0.9), label='Alto Potencial (>15)'),
    Patch(facecolor=plt.cm.RdYlGn(0.6), label='Médio Potencial (12-15)'),
    Patch(facecolor=plt.cm.RdYlGn(0.3), label='Baixo Potencial (<12)')
]
ax4.legend(handles=legend_elements, loc='upper right', fontsize=9)

plt.tight_layout()
plt.savefig(("../../assets/gráficos/potencial_clusters.png"), dpi=300, bbox_inches='tight')
plt.show()

# RECOMENDAÇÕES DETALHADAS PARA NOVOS LOCAIS
print(f"\n{'='*70}")
print("RECOMENDAÇÕES DE LOCAIS IDEAIS PARA NOVAS LOJAS")
print(f"{'='*70}")

# Ordenando clusters por potencial
cluster_df_ordenado = cluster_df.sort_values('potencial_medio', ascending=False)

for idx, cluster_info in cluster_df_ordenado.iterrows():
    cluster_id = int(cluster_info['cluster'] - 1)  # Ajuste do índice
    print(f"\n🏪 PRIORIDADE {idx+1} - CLUSTER {cluster_info['cluster']}:")
    print(f"   📍 Localização ideal: ({cluster_info['centro_lat']:.2f}, {cluster_info['centro_lon']:.2f})")
    print(f"   📊 Potencial médio: {cluster_info['potencial_medio']:.2f}")
    print(f"   🏢 Regiões no cluster: {cluster_info['num_regioes']}")
    print(f"   👓 Vendas óculos de grau: {cluster_info['vendas_oculos_grau_total']}")
    print(f"   👥 Base de clientes: {cluster_info['num_clientes_total']:,}")
    print(f"   💰 Receita total: R$ {cluster_info['receita_total']:,.2f}")
    
    # Encontrando as 3 melhores regiões dentro do cluster
    cluster_data = dados_expansao_outros[clusters_expansao == cluster_id]
    top_regioes = cluster_data.nlargest(3, 'potencial_score')
    
    print(f"   🎯 TOP 3 REGIÕES RECOMENDADAS:")
    for j, (region_idx, regiao) in enumerate(top_regioes.iterrows()):
        print(f"      {j+1}. {regiao['estado']} (Região {regiao['cod_cidade']}):")
        print(f"         - Score: {regiao['potencial_score']:.2f}")
        print(f"         - Vendas óculos: {regiao['vendas_oculos_grau']}")
        print(f"         - Lojas atuais: {regiao['num_lojas']}")
        print(f"         - Clientes: {regiao['num_clientes']:,}")
        
        # Calculando distância do centróide
        distancia_centroide = haversine_distance(
            cluster_info['centro_lat'], cluster_info['centro_lon'],
            regiao['latitude'], regiao['longitude']
        )
        print(f"         - Distância do ponto ideal: {distancia_centroide:.1f} km")

print(f"\n{'='*70}")
print("RESUMO EXECUTIVO:")
print(f"{'='*70}")
print(cluster_df_ordenado[['cluster', 'potencial_medio', 'vendas_oculos_grau_total', 
                         'num_clientes_total', 'receita_total']].to_string(index=False))

## Exportação de Sugestões para o Modelo Supervisionado

Esta seção representa a **fase final e crítica do modelo não supervisionado**, onde todos os insights e análises anteriores são consolidados e transformados em um dataset estruturado e acionável para alimentar o modelo supervisionado subsequente.

**1. Determinação dos Valores K Ideais:**
- **Validação dos parâmetros ótimos**: Confirma os melhores valores de k identificados nas análises de silhouette
- **São Paulo (k=2)**: Configuração otimizada para características metropolitanas específicas
- **Outros Estados (k=3)**: Segmentação nacional balanceada para diversidade regional
- **Fallback inteligente**: Sistema de recuperação caso variáveis não estejam disponíveis

**2. Recalculagem de Clusters com Parâmetros Otimizados:**
- **Clustering final de SP**: Aplicação do K-Means com k=2 usando sample weights baseados em vendas de óculos de grau
- **Clustering nacional**: Segmentação de outros estados com k=3 priorizando score de potencial
- **Centróides precisos**: Identificação matemática dos pontos ideais para cada cluster
- **Validação de qualidade**: Confirmação da criação bem-sucedida de todos os clusters

**3. Sistema de Geração de Sugestões Inteligente:**
- **Processamento por escopo**: Análise separada e especializada para SP vs outros estados
- **Mapeamento centróide-realidade**: Conversão de pontos matemáticos abstratos em localizações geográficas reais
- **Função Haversine integrada**: Cálculo de distâncias reais entre centróides e regiões existentes
- **Proteção contra dados vazios**: Validações robustas para evitar erros em clusters sem dados

**4. Métricas Abrangentes por Sugestão:**
- **Coordenadas duais**: Tanto centróides teóricos quanto localizações reais sugeridas
- **Score de potencial**: Médio e máximo por cluster para priorização estratégica
- **Métricas de performance**: Vendas de óculos de grau, clientes, receita e número de lojas
- **Indicadores normalizados**: Métricas por loja para comparação justa entre regiões

**5. Estrutura de Dados Padronizada:**
- **Campos consistentes**: Esquema uniforme para todas as sugestões independente do escopo
- **Tipos de dados otimizados**: Conversões apropriadas (int, float, string) para eficiência
- **Informações contextuais**: Estado, região Chilli Beans e classificação de escopo
- **Distâncias validadas**: Proximidade em quilômetros entre centróides e sugestões

**6. Sistema de Priorização Hierárquica:**
- **Ranking por escopo**: Ordenação dentro de São Paulo e outros estados separadamente
- **Score como critério**: Potencial médio do cluster determina ordem de prioridade
- **Numeração automática**: Prioridades de 1 a N dentro de cada escopo geográfico
- **Flexibilidade estratégica**: Permite abordagens diferenciadas por região

**7. Validação e Proteção de Dados:**
- **Verificação de existência**: Confirma disponibilidade de todas as variáveis necessárias
- **Tratamento de divisão por zero**: Proteção em normalizações por número de lojas
- **Dados ausentes**: Fallbacks apropriados para cenários de dados incompletos
- **Consistência de tipos**: Garantia de que todos os campos estão nos formatos corretos

**8. Exportação Estruturada para CSV:**
- **Arquivo padronizado**: `sugestoes_expansao_para_supervisionado.csv` com esquema definido
- **Compatibilidade garantida**: Formato ideal para ingestão por modelos de machine learning
- **Metadados incluídos**: Informações suficientes para interpretação independente
- **Versionamento implícito**: Dados datados e reproduzíveis para auditoria

**9. Campos Exportados Detalhados:**
- **Identificação**: escopo, cluster_id, prioridade_no_escopo
- **Coordenadas**: latitude/longitude de centróides e sugestões
- **Geografia**: estado, região Chilli Beans
- **Performance**: vendas, clientes, receita (total e por loja)
- **Qualidade**: potencial score, distância do centróide

**10. Estatísticas de Validação:**
- **Contagem por escopo**: Número de sugestões geradas para SP vs outros estados
- **Potencial médio**: Score agregado para validar qualidade das recomendações
- **Vendas totais**: Soma das vendas de óculos de grau nas regiões sugeridas
- **Shape do dataset**: Dimensões finais para confirmação de completude

**11. Integração com Modelo Supervisionado:**
- **Features prontas**: Variáveis numéricas e categóricas estruturadas para ML
- **Target implícito**: Score de potencial pode servir como variável de interesse
- **Coordenadas para geocoding**: Dados geográficos para análises espaciais avançadas
- **Contexto de negócio**: Informações interpretáveis para stakeholders

**12. Benefícios da Abordagem:**
- **Redução de processamento**: Modelo supervisionado recebe dados pré-processados e validados
- **Reprodutibilidade**: Mesmos dados de entrada geram mesmas sugestões
- **Escalabilidade**: Framework aplicável a futuras expansões ou atualizações
- **Auditabilidade**: Rastro completo desde dados brutos até recomendações finais

**13. Outputs de Qualidade:**
- **Dataset limpo**: Sem valores nulos ou inconsistências
- **Distribuição balanceada**: Sugestões cobrindo diferentes regiões e potenciais
- **Métricas validadas**: Todos os indicadores dentro de ranges esperados
- **Pronto para produção**: Formato diretamente utilizável por sistemas downstream

**14. Preparação para Decisões Estratégicas:**
- **Priorização clara**: Ordem de investimento baseada em dados objetivos
- **ROI estimado**: Métricas de potencial fundamentam expectativas de retorno
- **Diversificação geográfica**: Mix de oportunidades em diferentes mercados
- **Implementação faseada**: Permite execução gradual baseada em prioridades

Esta etapa **converte análises exploratórias em ações concretas**, fornecendo ao modelo supervisionado uma base sólida de localizações pré-qualificadas e priorizadas, maximizando a eficiência e precisão das próximas fases do projeto de expansão da Chilli Beans com foco em óculos

In [ ]:
# AVALIAÇÃO E INTERPRETAÇÃO DOS RESULTADOS

print("="*80)
print("MÉTRICAS DE QUALIDADE DA CLUSTERIZAÇÃO")
print("="*80)

# Avaliação das métricas para diferentes valores de k
fig, axes = plt.subplots(2, 2, figsize=(20, 12))

# Comparação de inércia entre SP e outros estados
ax1 = axes[0, 0]
line_sp = ax1.plot(k_values_sp, inertias_sp, 'ro-', label='São Paulo', linewidth=2, markersize=6)
line_outros = ax1.plot(k_values_outros[:len(inertias_outros)], inertias_outros, 'bo-', label='Outros Estados', linewidth=2, markersize=6)
ax1.set_xlabel('Número de Clusters (k)')
ax1.set_ylabel('Inércia (WCSS)')
ax1.set_title('Comparação da Inércia: SP vs Outros Estados\n(Menor inércia = Melhor agrupamento)')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)
# Adicionar anotação explicativa
ax1.text(0.02, 0.98, 'Inércia mede a compactação dos clusters\n(valores menores são melhores)', 
         transform=ax1.transAxes, fontsize=9, 
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightyellow", alpha=0.8),
         verticalalignment='top')

# Comparação de Silhouette Score
ax2 = axes[0, 1]
if silhouette_scores_sp:
    ax2.plot(k_values_sp[1:], silhouette_scores_sp, 'ro-', label='São Paulo', linewidth=2, markersize=6)
if silhouette_scores_outros:
    ax2.plot(k_values_outros[1:len(silhouette_scores_outros)+1], silhouette_scores_outros, 'bo-', label='Outros Estados', linewidth=2, markersize=6)
ax2.set_xlabel('Número de Clusters (k)')
ax2.set_ylabel('Silhouette Score')
ax2.set_title('Comparação do Silhouette Score\n(Maior score = Melhor separação)')
ax2.legend(loc='lower right')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, max(max(silhouette_scores_outros) if silhouette_scores_outros else 0, 
                   max(silhouette_scores_sp) if silhouette_scores_sp else 0) + 0.1)
# Adicionar linha de referência para qualidade boa (>0.5)
ax2.axhline(y=0.5, color='green', linestyle='--', alpha=0.7, label='Boa qualidade (>0.5)')
ax2.legend(loc='lower right')

# Distribuição de vendas por região
ax3 = axes[1, 0]
regioes = ['São Paulo', 'Outros Estados']
vendas_por_regiao = [dados_sp['vendas_oculos_grau'].sum(), dados_outros_estados['vendas_oculos_grau'].sum()]
colors = ['red', 'blue']
wedges, texts, autotexts = ax3.pie(vendas_por_regiao, labels=regioes, colors=colors, 
                                   autopct='%1.1f%%', startangle=90, 
                                   explode=(0.05, 0))  # Destacar São Paulo
ax3.set_title('Distribuição de Vendas de Óculos de Grau por Região')
# Melhorar formatação dos textos
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')
    autotext.set_fontsize(12)
# Adicionar valores absolutos
total_vendas = sum(vendas_por_regiao)
ax3.text(0.02, -1.3, f'Total de vendas: {total_vendas:,} unidades', 
         transform=ax3.transData, fontsize=10, 
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgray", alpha=0.8))

# Eficiência do modelo - Relação lojas vs vendas
ax4 = axes[1, 1]
lojas_por_regiao = [dados_sp['num_lojas'].sum(), dados_outros_estados['num_lojas'].sum()]
eficiencia = [v/l for v, l in zip(vendas_por_regiao, lojas_por_regiao)]
bars = ax4.bar(regioes, eficiencia, color=colors, alpha=0.7, width=0.6)
ax4.set_ylabel('Vendas de Óculos por Loja (unidades)')
ax4.set_title('Eficiência de Vendas por Região\n(Vendas de óculos por loja)')
ax4.grid(True, alpha=0.3, axis='y')
ax4.set_ylim(0, max(eficiencia) * 1.2)

# Adicionando valores nas barras com mais detalhes
for i, (bar, valor, vendas, lojas) in enumerate(zip(bars, eficiencia, vendas_por_regiao, lojas_por_regiao)):
    ax4.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.05, 
             f'{valor:.2f}', ha='center', va='bottom', fontweight='bold', fontsize=12)
    # Adicionar informações adicionais
    ax4.text(bar.get_x() + bar.get_width()/2., -0.15, 
             f'{vendas} vendas\n{lojas} lojas', ha='center', va='top', fontsize=9,
             bbox=dict(boxstyle="round,pad=0.2", facecolor=colors[i], alpha=0.3))

# Adicionar linha de média
media_eficiencia = sum(vendas_por_regiao) / sum(lojas_por_regiao)
ax4.axhline(y=media_eficiencia, color='green', linestyle='--', alpha=0.7, 
           label=f'Média geral: {media_eficiencia:.2f}')
ax4.legend(loc='upper right')

plt.tight_layout()
plt.savefig(("../../assets/gráficos/comparacoes.png"), dpi=300, bbox_inches='tight')
plt.show()

# INSIGHTS ESTRATÉGICOS
print("\n" + "="*80)
print("INSIGHTS ESTRATÉGICOS PARA EXPANSÃO DA CHILLI BEANS")
print("="*80)

# Calculando métricas-chave
total_vendas_oculos = dados_localizacao['vendas_oculos_grau'].sum()
total_lojas = dados_localizacao['num_lojas'].sum()
total_clientes = dados_localizacao['num_clientes'].sum()
total_receita = dados_localizacao['receita_total'].sum()

participacao_sp = (dados_sp['vendas_oculos_grau'].sum() / total_vendas_oculos) * 100
participacao_outros = (dados_outros_estados['vendas_oculos_grau'].sum() / total_vendas_oculos) * 100

print(f"\n📊 PANORAMA GERAL:")
print(f"   • Total de vendas de óculos de grau: {total_vendas_oculos:,}")
print(f"   • Total de lojas: {total_lojas:,}")
print(f"   • Total de clientes: {total_clientes:,}")
print(f"   • Receita total: R$ {total_receita:,.2f}")
print(f"   • Média de vendas por loja: {total_vendas_oculos/total_lojas:.2f}")

print(f"\n🗺️ DISTRIBUIÇÃO REGIONAL:")
print(f"   • São Paulo: {participacao_sp:.1f}% das vendas de óculos de grau")
print(f"   • Outros Estados: {participacao_outros:.1f}% das vendas de óculos de grau")
print(f"   • Concentração em SP justifica análise separada ✓")

print(f"\n🎯 RECOMENDAÇÕES PARA NOVAS LOJAS:")

# Top 5 estados com maior potencial (excluindo SP) - usando dados_expansao_outros que tem potencial_score
top_estados_potencial = dados_expansao_outros.groupby('estado').agg({
    'potencial_score': 'mean',
    'vendas_oculos_grau': 'sum',
    'num_lojas': 'sum',
    'num_clientes': 'sum'
}).sort_values('potencial_score', ascending=False).head(5)

print(f"\n   🏆 TOP 5 ESTADOS COM MAIOR POTENCIAL (excluindo SP):")
for i, (estado, dados_estado) in enumerate(top_estados_potencial.iterrows(), 1):
    print(f"      {i}. {estado}:")
    print(f"         - Score médio: {dados_estado['potencial_score']:.2f}")
    print(f"         - Vendas atuais: {dados_estado['vendas_oculos_grau']}")
    print(f"         - Lojas existentes: {dados_estado['num_lojas']}")
    print(f"         - Base de clientes: {dados_estado['num_clientes']:,}")

print(f"\n🔍 ANÁLISE DE CLUSTERS ÓTIMOS:")
print(f"   • São Paulo: Melhor k = {best_k_sp if 'best_k_sp' in locals() else 'N/A'}")
print(f"   • Outros Estados: Melhor k = {best_k_outros}")
print(f"   • Silhouette Score ótimo: {max(silhouette_scores_outros):.3f}")

print(f"\n💡 INSIGHTS DE NEGÓCIO:")
print(f"   1. 🎯 FOCO GEOGRÁFICO:")
print(f"      • Priorizar expansão nos clusters com maior potencial")
print(f"      • Manter estratégia diferenciada para São Paulo")
print(f"      • Explorar oportunidades no Nordeste e Sul")

print(f"\n   2. 📈 ESTRATÉGIA DE PRODUTOS:")
print(f"      • Óculos de grau representam mercado estratégico")
print(f"      • Considerar mix de produtos por região")
print(f"      • Focar em regiões com alta base de clientes")

print(f"\n   3. 🏪 LOCALIZAÇÃO DE LOJAS:")
print(f"      • Usar centróides dos clusters como referência")
print(f"      • Considerar densidade populacional local")
print(f"      • Avaliar concorrência em cada região")

print(f"\n   4. 📊 MÉTRICAS DE ACOMPANHAMENTO:")
print(f"      • Vendas de óculos de grau por loja")
print(f"      • Penetração de mercado por cluster")
print(f"      • ROI por região de expansão")

# Resumo dos clusters identificados
print(f"\n📍 RESUMO DOS CLUSTERS PARA EXPANSÃO:")
if 'cluster_df' in locals():
    cluster_resumo = cluster_df.sort_values('potencial_medio', ascending=False)
    for idx, row in cluster_resumo.iterrows():
        prioridade = "🥇 ALTA" if row['potencial_medio'] > 15 else "🥈 MÉDIA" if row['potencial_medio'] > 12 else "🥉 BAIXA"
        print(f"   Cluster {int(row['cluster'])}: {prioridade} PRIORIDADE")
        print(f"      - Potencial: {row['potencial_medio']:.1f}")
        print(f"      - Localização: ({row['centro_lat']:.1f}, {row['centro_lon']:.1f})")

print(f"\n" + "="*80)
print("CONCLUSÃO: O modelo identificou 5 clusters regionais ótimos para expansão,")
print("com oportunidades claras de crescimento focadas em óculos de grau.")
print("A abordagem separada SP vs outros estados evitou viés e forneceu")
print("insights estratégicos mais precisos para a tomada de decisão.")
print("="*80)

## 🔄 Correção dos Valores de *k* por Região (Recalibração de Clusters)

Esta célula realiza uma **recalibração explícita do número de clusters (k)** para cada recorte geográfico (São Paulo vs Outros Estados), alinhando o pipeline aos resultados observados nos gráficos anteriores de **Curva do Cotovelo (Elbow)** e, secundariamente, ao comportamento do **Silhouette Score**.

**1. Contexto da Correção**  
- Etapas anteriores testaram múltiplos valores de k (1–6) para cada região  
- Visualmente, a Curva do Cotovelo indicou estabilização mais precoce em SP (k≈2) e leve ganho adicional nacional em k≈3  
- Esta célula força o uso desses valores “ótimos operacionais” para padronizar as etapas subsequentes

**2. Motivo da Separação SP vs Outros**  
- São Paulo tem densidade e concentração superiores → menor k evita subdivisão artificial  
- Outros Estados apresentam maior dispersão geográfica → k=3 captura macro-regiões sem superfragmentar

**3. Critérios Utilizados para Escolha**  
- Elbow: ponto onde a redução da inércia passa a ter retorno marginal  
- Silhouette: verificação de separação aceitável entre grupos (valores > ~0.3 já indicam estrutura mínima; >0.5 seria ideal)  
- Trade-off: simplicidade + interpretabilidade > overfitting espacial

**4. Ações Executadas na Célula**  
1. Define explicitamente: `best_k_sp = 2` e `best_k_outros = 3`  
2. Reinstancia dois modelos `KMeans` independentes  
3. Recalcula rótulos de cluster (`clusters_sp`, `clusters_outros`)  
4. Recalcula coeficientes de silhueta para validação rápida

**5. Métricas Calculadas**  
- `silhouette_sp_correto`: qualidade da separação para SP com k=2  
- `silhouette_outros_correto`: qualidade da separação nacional (exceto SP) com k=3  
- Uso direto de latitude/longitude sem pesos nesta etapa (foco: validação estrutural simples)

**6. Benefícios da Recalibração**  
- Reduz risco de supersegmentação em áreas densas  
- Mantém clusters interpretáveis para relato executivo  
- Melhora consistência entre etapas exploratórias e etapas de exportação/decisão

**7. Impacto no Pipeline Posterior**  
- Afeta qualquer etapa que:  
  - Usa `clusters_sp` / `clusters_outros` para agregação ou ranking  
  - Gera centróides para recomendações geográficas  
  - Calcula estatísticas comparativas por cluster  
- Garante coerência com exportação final das sugestões supervisionadas

**8. Limitações / Alertas**  
- Não recalcula pesos por vendas/potencial (versão simplificada)  
- Silhouette em coordenadas puras não considera intensidade de negócio  
- Caso a distribuição futura mude (novas lojas), valores de k devem ser revisados

**9. Possíveis Extensões Futuras**  
- Repetir análise com clustering ponderado (sample_weight)  
- Testar algoritmos alternativos: DBSCAN (densidade) ou HDBSCAN (robusto a ruído)  
- Incluir features enriquecidas (potencial_score normalizado, densidade de clientes)

**10. Interpretação dos Resultados Impressos**  
- Linhas de saída confirmam k aplicado para cada região  
- Exibem silhueta final → validação rápida de separação espacial  
- Mensagem final dá o resumo operacional adotado

**11. Quando Reexecutar Esta Célula**  
- Após alteração nas coordenadas ou filtragem geográfica  
- Após redefinição de pesos estratégicos ou normalização espacial  
- Antes de exportar novos CSVs de sugestões supervisionadas

**12. Valor Estratégico**  
- Consolida escolha de segmentação espacial “oficial”  
- Serve como checkpoint de governança analítica  
- Evita divergência entre experimentação e entrega executiva

**Resumo:** Esta célula “congela” as decisões de k (SP=2, Outros=3) com base nas análises anteriores e revalida rapidamente a qualidade dos agrupamentos para garantir consistência e interpretabilidade nas etapas finais do

In [ ]:
# CORREÇÃO: k específico para cada região baseado no método do cotovelo
print("=== CORREÇÃO DOS VALORES K BASEADO NA ANÁLISE ESPECÍFICA DO COTOVELO ===")
print("São Paulo: k=2 (conforme gráfico do cotovelo)")
print("Outros Estados: k=3 (conforme gráfico do cotovelo)")
print("\nAtualizando os valores...")

# Redefinir os valores de k baseado na análise específica do cotovelo
best_k_sp = 2
best_k_outros = 3

print(f"K atualizado para São Paulo: {best_k_sp}")
print(f"K atualizado para Outros Estados: {best_k_outros}")

# Recalcular clustering com os valores corretos
print("\nRecalculando clustering com valores específicos...")

# São Paulo com k=2
kmeans_sp = KMeans(n_clusters=best_k_sp, random_state=42, n_init=10)
clusters_sp = kmeans_sp.fit_predict(X_sp)

# Outros Estados com k=3
kmeans_outros = KMeans(n_clusters=best_k_outros, random_state=42, n_init=10)
clusters_outros = kmeans_outros.fit_predict(X_outros)

# Recalcular silhueta com os novos valores
from sklearn.metrics import silhouette_score
silhouette_sp_correto = silhouette_score(X_sp, clusters_sp)
silhouette_outros_correto = silhouette_score(X_outros, clusters_outros)

print(f"\nCoeficientes de silhueta com valores corretos:")
print(f"São Paulo (k=2): {silhouette_sp_correto:.3f}")
print(f"Outros Estados (k=3): {silhouette_outros_correto:.3f}")

print("\nClustering corrigido com sucesso!")
print(f"\nResumo:")
print(f"• São Paulo será dividido em {best_k_sp} clusters")
print(f"• Outros Estados serão divididos em {best_k_outros} clusters")

## Métrica Pseudo R² e Verificação do Ambiente de Clusters (Célula 31)

Esta célula introduz uma métrica adicional de qualidade para os agrupamentos (pseudo R²), além de executar verificações de integridade das variáveis de clustering já criadas, garantindo que o ambiente esteja consistente antes de análises posteriores ponderadas.

**1. Objetivo Principal**  
- Calcular uma medida tipo “fração de variância explicada” pelos clusters (pseudo R²)  
- Confirmar a existência e formato de variáveis críticas (`X_sp`, `X_outros`, `clusters_sp`, `clusters_outros`)  
- Manter compatibilidade com mudanças de nomenclatura (uso atual de `vendas_oculos_grau`)

**2. Contexto no Pipeline**  
- Após definir e recalibrar k para cada região, já existem rótulos de cluster  
- Silhouette já foi usado; esta métrica complementa a avaliação com uma visão “variância explicada”  
- Serve como diagnóstico rápido sem gerar gráficos adicionais

**3. Ajuste de Terminologia**  
- Mensagem inicial esclarece que antigas referências genéricas a “vendas” foram alinhadas ao nome consolidado: `vendas_oculos_grau`  
- Evita confusão em logs antigos ou comparações de execução

**4. Função `calcular_pseudo_r2()`**  
- Fórmula: pseudo_r2 = 1 − (SSE_within / SST_total)  
  - SST: soma dos quadrados das distâncias de cada ponto ao centro global  
  - SSE: soma dos quadrados das distâncias de cada ponto ao centróide do seu cluster  
- Interpretação: quanto maior (próximo de 1), maior proporção da variabilidade espacial capturada pelos agrupamentos  
- Sempre entre 0 e 1 (SSE ≤ SST em partições não degeneradas)

**5. Passos Executados**  
1. Imprime médias de vendas de óculos de grau (SP e outros)  
2. Define função de pseudo R² (uso de norma Euclidiana em latitude/longitude)  
3. Lista variáveis esperadas e suas shapes (debug de ambiente)  
4. Tenta calcular `pseudo_r2_sp` e `pseudo_r2_outros` com tratamento de exceções  
5. Armazena `None` caso variáveis não existam ou ocorram erros

**6. Interpretação do Pseudo R²**  
- Valores mais altos: clusters capturam estrutura espacial (boa separação macro)  
- Valores modestos: dispersão geográfica elevada ou k pequeno (intencional para interpretabilidade)  
- Comparável apenas dentro do mesmo espaço de features (não misturar com métricas ponderadas futuras)

**7. Benefícios da Métrica**  
- Simples, interpretável e rápida de calcular  
- Complementa Silhouette (separação relativa) com visão de “redução de variância”  
- Útil em relatórios executivos como indicação percentual (ex.: “clusters explicam ~X% da variabilidade espacial”)

**8. Limitações / Alertas**  
- Usa coordenadas em plano (não corrige curvatura da Terra) → aceitável para escala nacional aproximada  
- Não utiliza pesos (ignora importância comercial/local)  
- Não considera densidade ou forma não esférica dos clusters (K-Means impõe convexidade)  
- Não substitui métricas padrão (Silhouette, Calinski-Harabasz, Davies-Bouldin)

**9. Possíveis Extensões**  
- Versão ponderada: incluir `sample_weight` replicando pontos ou ajustando SSE  
- Adotar distância Haversine real (converter lat/lon em radianos)  
- Comparar com proporção Between / Total (BSS / TSS) explicitando variância entre clusters  
- Calcular conjuntamente Calinski-Harabasz para reforço analítico

**10. Como Validar Resultados**  
- Verificar se os valores não são 0 ou muito próximos de 0 (indica ausência de estrutura)  
- Comparar SP vs Outros: SP tende a ter pseudo R² menor se altamente concentrado em poucas áreas, ou maior se clusters bem definidos  
- Reexecutar após alterar k ou redefinir subset geográfico

**11. Impacto no Fluxo Posterior**  
- Pode embasar justificativa de manutenção de k baixo (trade-off simplicidade vs ganho marginal)  
- Referência adicional nas análises ponderadas subsequentes  
- Não altera dados — apenas diagnósticos

**12. Quando Reexecutar**  
- Após recalcular clusters com novo k  
- Após filtragens geográficas ou remoção de outliers  
- Antes de gerar relatório final comparativo de métricas

**Resumo:** A célula adiciona uma métrica complementar (pseudo R²) para avaliar a fração de variabilidade espacial capturada pelos clusters e valida a presença das variáveis de clustering em memória, fortalecendo a governança analítica antes de análises ponderadas

In [ ]:
# Nota: algumas saídas antigas do notebook faziam referência a 'vendas'. Elas foram atualizadas para os nomes atuais.
# Versão segura das mensagens antigas:
print(f"  Vendas médias por região: {dados_sp['vendas_oculos_grau'].mean():.0f}")
print(f"  Vendas médias por região: {dados_outros_estados['vendas_oculos_grau'].mean():.0f}")


def calcular_pseudo_r2(X, clusters):
    """
    Calcula uma medida tipo pseudo R² (fração da variância explicada) para uma partição em clusters.
    pseudo_r2 = 1 - (SSE_within / SST_total), onde SSE_within é a soma dos quadrados dentro dos clusters
    e SST_total é soma dos quadrados total em relação à média global.
    """
    import numpy as _np

    # centroides globais
    global_mean = _np.mean(X, axis=0)
    # SST total
    sst = _np.sum((_np.linalg.norm(X - global_mean, axis=1))**2)

    # SSE within
    sse = 0.0
    for k in _np.unique(clusters):
        mask = clusters == k
        if mask.sum() == 0:
            continue
        centroid = _np.mean(X[mask], axis=0)
        sse += _np.sum((_np.linalg.norm(X[mask] - centroid, axis=1))**2)

    if sst == 0:
        return 0.0
    return 1 - (sse / sst)

# Debug: verificar variáveis disponíveis
print("Variáveis de clustering disponíveis:")
for var_name in ['X_sp', 'X_outros', 'clusters_sp', 'clusters_outros']:
    if var_name in globals():
        print(f"  {var_name}: {type(globals()[var_name])}, shape: {getattr(globals()[var_name], 'shape', 'N/A')}")
    else:
        print(f"  {var_name}: não encontrado")

# Calcular pseudo-R2 não ponderado usando variáveis disponíveis
try:
    if 'X_sp' in globals() and 'clusters_sp' in globals():
        pseudo_r2_sp = calcular_pseudo_r2(X_sp, clusters_sp)
        print(f"Calculado pseudo_r2_sp: {pseudo_r2_sp}")
    else:
        pseudo_r2_sp = None
        print("Não foi possível calcular pseudo_r2_sp - variáveis necessárias não encontradas")
except Exception as e:
    pseudo_r2_sp = None
    print(f"Erro ao calcular pseudo_r2_sp: {e}")

try:
    if 'X_outros' in globals() and 'clusters_outros' in globals():
        pseudo_r2_outros = calcular_pseudo_r2(X_outros, clusters_outros)
        print(f"Calculado pseudo_r2_outros: {pseudo_r2_outros}")
    else:
        pseudo_r2_outros = None
        print("Não foi possível calcular pseudo_r2_outros - variáveis necessárias não encontradas")
except Exception as e:
    pseudo_r2_outros = None
    print(f"Erro ao calcular pseudo_r2_outros: {e}")

# Função ponderada já existente será chamada posteriormente

## Análise de Correlação entre Clusters e Variáveis de Negócio

Esta célula executa uma análise exploratória para identificar associações lineares entre os rótulos de cluster gerados pelo algoritmo K-Means e as principais métricas de performance comercial da rede Chilli Beans, fornecendo insights sobre como a segmentação geográfica se relaciona com o desempenho operacional.

**1. Objetivo da Análise**  
Verificar se existe correlação linear significativa entre a atribuição de clusters espaciais e as variáveis de negócio como vendas, número de clientes, receita e quantidade de lojas, permitindo avaliar se a segmentação geográfica captura naturalmente diferenças de performance comercial.

**2. Metodologia Aplicada**  
A célula utiliza o coeficiente de correlação de Pearson para medir a força e direção da relação linear entre o identificador numérico do cluster e cada métrica de negócio. O teste é executado separadamente para São Paulo e demais estados, mantendo a consistência com a abordagem de análise regionalizada.

**3. Preparação dos Dados**  
Implementa uma função de padronização que cria aliases automáticos para as colunas, mapeando os nomes reais das variáveis para nomenclaturas padronizadas. Esta funcionalidade garante compatibilidade e robustez na análise, convertendo automaticamente vendas_oculos_grau para vendas e vendas_oculos, num_clientes para clientes, receita_total para receita, e num_lojas para lojas.

**4. Processo de Cálculo**  
Para cada região, a célula itera através das métricas padronizadas, verificando a existência da coluna no dataset e a presença de variação suficiente nos dados. Calcula o coeficiente de correlação de Pearson e o respectivo p-value, armazenando os resultados em dicionários estruturados para posterior consolidação.

**5. Validações Implementadas**  
Inclui verificações de integridade que identificam situações onde a correlação não pode ser calculada, como ausência de variação nos dados ou clusters com cardinalidade insuficiente. Estas validações previnem erros matemáticos e informam sobre limitações dos dados.

**6. Resultados Consolidados**  
Gera um DataFrame consolidado com todas as correlações calculadas, organizando os resultados por escopo geográfico e métrica, facilitando a interpretação comparativa. Exibe tanto os resultados individuais quanto um resumo tabular estruturado.

**7. Interpretação Estatística**  
Fornece diretrizes claras para interpretação dos resultados, classificando correlações próximas de zero como ausência de relação linear, valores entre 0.3 e -0.3 como relação moderada a forte, e p-values menores que 0.05 como estatisticamente significativos.

**8. Aplicação Estratégica**  
Os resultados desta análise informam se os clusters atuais já se alinham naturalmente com métricas de performance ou se são puramente espaciais, orientando decisões sobre a necessidade de enriquecimento de features para versões futuras do modelo de clustering.

**9. Limitações Metodológicas**  
Reconhece que clusters são variáveis categóricas tratadas como numéricas para fins de correlação linear, sendo esta uma aproximação exploratória válida mas com limitações interpretativas. A análise assume linearidade e não captura relações monotônicas não-lineares que poderiam ser detectadas por correlação de Spearman.

**10. Valor Analítico**  
Serve como checkpoint rápido para avaliar o alinhamento entre segmentação geográfica e performance comercial, fornecendo evidências quantitativas para justificar modificações futuras na estratégia de clustering ou na seleção de features para o modelo de expansão.

In [ ]:
import scipy.stats as stats
from scipy.stats import pearsonr, spearmanr

print("=== TESTE 1: CORRELAÇÃO COM VARIÁVEIS EXTERNAS ===\n")

def padronizar_metricas(df):
    """Cria aliases padronizados para as colunas alvo, se existirem."""
    df_copia = df.copy()
    
    # Mapear nomes reais para aliases padronizados
    mapping = {
        'vendas_oculos_grau': ['vendas_oculos', 'vendas'],
        'num_clientes': ['clientes'],
        'receita_total': ['receita'],
        'num_lojas': ['lojas']
    }
    
    for origem, aliases in mapping.items():
        if origem in df_copia.columns:
            for alias in aliases:
                if alias not in df_copia.columns:
                    df_copia[alias] = df_copia[origem]
    
    return df_copia

# Para São Paulo
print("CORRELAÇÕES - SÃO PAULO:")
df_sp_test = padronizar_metricas(dados_sp)
df_sp_test['cluster'] = clusters_sp

# Correlação entre clusters e variáveis numéricas
correlacoes_sp = {}
metricas_teste = ['vendas', 'vendas_oculos', 'clientes', 'receita', 'lojas']

correlacoes_encontradas_sp = False
for col in metricas_teste:
    if col in df_sp_test.columns:
        # Verificar se há variação suficiente
        if df_sp_test[col].nunique() < 2 or df_sp_test['cluster'].nunique() < 2:
            print(f"  {col}: variância insuficiente (pulando)")
            continue
            
        corr_pearson, p_value = pearsonr(df_sp_test['cluster'], df_sp_test[col])
        correlacoes_sp[col] = {'correlacao': corr_pearson, 'p_value': p_value}
        print(f"  {col}: r = {corr_pearson:.3f}, p = {p_value:.3f}")
        correlacoes_encontradas_sp = True

if not correlacoes_encontradas_sp:
    print("  (Nenhuma métrica compatível encontrada)")

print("\nCORRELAÇÕES - OUTROS ESTADOS:")
df_outros_test = padronizar_metricas(dados_outros_estados)
df_outros_test['cluster'] = clusters_outros

# Correlação entre clusters e variáveis numéricas
correlacoes_outros = {}
correlacoes_encontradas_outros = False

for col in metricas_teste:
    if col in df_outros_test.columns:
        # Verificar se há variação suficiente
        if df_outros_test[col].nunique() < 2 or df_outros_test['cluster'].nunique() < 2:
            print(f"  {col}: variância insuficiente (pulando)")
            continue
            
        corr_pearson, p_value = pearsonr(df_outros_test['cluster'], df_outros_test[col])
        correlacoes_outros[col] = {'correlacao': corr_pearson, 'p_value': p_value}
        print(f"  {col}: r = {corr_pearson:.3f}, p = {p_value:.3f}")
        correlacoes_encontradas_outros = True

if not correlacoes_encontradas_outros:
    print("  (Nenhuma métrica compatível encontrada)")

# Consolidação opcional
import pandas as pd
resultados_correlacao = []
for col, dados in correlacoes_sp.items():
    resultados_correlacao.append({
        'escopo': 'SP',
        'metrica': col,
        'r': dados['correlacao'],
        'p_value': dados['p_value']
    })

for col, dados in correlacoes_outros.items():
    resultados_correlacao.append({
        'escopo': 'OUTROS',
        'metrica': col,
        'r': dados['correlacao'],
        'p_value': dados['p_value']
    })

if resultados_correlacao:
    df_corr = pd.DataFrame(resultados_correlacao)
    print("\nResumo consolidado:")
    print(df_corr.round(3))

print("\nINTERPRETAÇÃO:")
print("- Correlações próximas de 0: clusters não relacionados linearmente com a variável")
print("- Correlações > 0.3 ou < -0.3: relação moderada a forte")
print("- p < 0.05: correlação estatisticamente significativa")

## Checkpoint Estrutural de Métricas

Esta célula atua como a primeira barreira de controle antes de qualquer derivação analítica adicional. O foco é exclusivamente estrutural: garantir que as tabelas segmentadas (`dados_sp`, `dados_outros_estados`) contêm as colunas e linhas mínimas necessárias para prosseguir.

**Escopo**  
Valida presença e legibilidade de colunas núcleo sem fazer cálculos derivados ou normalizações.

**Colunas Verificadas (se existirem):**  
- `vendas_oculos_grau` (e fallback: `vendas_oculos`, `vendas`)  
- `num_clientes`  
- `receita_total`  
- `num_lojas`

**Operações Executadas**  
1. Testa existência de `dados_sp` e `dados_outros_estados` no ambiente.  
2. Calcula médias simples de vendas por região quando a coluna correspondente está presente.  
3. Cria um recorte de exemplo (`iloc[0:1]`) para demonstrar acesso defensivo a um registro.  
4. Aplica hierarquia de fallback para coluna de vendas (prioridade: `vendas_oculos_grau` → `vendas_oculos` → `vendas`).  
5. Exibe somatórios básicos das demais métricas somente se existirem e suportarem agregação.

**Não Faz**  
- Normalização por loja  
- Consolidação de clusters  
- Avaliação de qualidade estatística  
- Exportação ou mutação de dados

**Critérios Mínimos de Continuidade**  
- Cada DataFrame regional contém ao menos 1 linha  
- Pelo menos uma coluna de vendas identificada  
- Acesso sem exceções às estruturas principais

**Risco Mitigado**  
Interrupção tardia do pipeline devido a ausência silenciosa de colunas essenciais após limpeza/transformação.

**Recomendação**  
Se algum item crítico estiver ausente aqui, interromper execução e revisar etapas de preparação antes de seguir para pseudo R², correlações ou ponderações.

Resumo: checkpoint inicial enxuto para validar integridade estrutural de insumos antes de cálculos analíticos aprofundados.

In [ ]:
# Exibição final de métricas (ajustada para nomes reais)
print("=== VERIFICAÇÃO DE MÉTRICAS FINAIS ===")

# Verificação de vendas médias
if 'vendas_oculos_grau' in dados_sp.columns:
    print(f"Vendas médias SP: {dados_sp['vendas_oculos_grau'].mean():.0f}")
else:
    print("Vendas médias SP: coluna não encontrada")

if 'vendas_oculos_grau' in dados_outros_estados.columns:
    print(f"Vendas médias Outros Estados: {dados_outros_estados['vendas_oculos_grau'].mean():.0f}")
else:
    print("Vendas médias Outros Estados: coluna não encontrada")

# Exemplo de verificação segura para dados de cluster
print("\nVerificação de dados de cluster (exemplo):")

# Verificar se existem dados de cluster em memória
if 'dados_sp' in globals() and len(dados_sp) > 0:
    # Criar um exemplo de cluster_data para demonstração
    cluster_data = dados_sp.iloc[0:1]  # Primeiro registro como exemplo
    
    print("Dados de exemplo do primeiro cluster:")
    
    # Tentar diferentes nomes de colunas para vendas
    vendas_col = None
    if 'vendas_oculos_grau' in cluster_data.columns:
        vendas_col = cluster_data['vendas_oculos_grau']
        nome_coluna = 'vendas_oculos_grau'
    elif 'vendas_oculos' in cluster_data.columns:
        vendas_col = cluster_data['vendas_oculos']
        nome_coluna = 'vendas_oculos'
    elif 'vendas' in cluster_data.columns:
        vendas_col = cluster_data['vendas']
        nome_coluna = 'vendas'
    
    if vendas_col is not None and hasattr(vendas_col, 'sum'):
        print(f"  Vendas totais ({nome_coluna}): {vendas_col.sum():,.0f}")
    else:
        print("  Vendas totais: (dados não disponíveis)")
    
    # Verificar outras métricas
    if 'num_clientes' in cluster_data.columns:
        print(f"  Clientes totais: {cluster_data['num_clientes'].sum():,.0f}")
    
    if 'receita_total' in cluster_data.columns:
        print(f"  Receita total: R$ {cluster_data['receita_total'].sum():,.2f}")
    
    if 'num_lojas' in cluster_data.columns:
        print(f"  Número de lojas: {cluster_data['num_lojas'].sum()}")

else:
    print("Dados de cluster não disponíveis para verificação")

print("\n=== VERIFICAÇÃO CONCLUÍDA ===")

## Auditoria Pós-Transformações e Consistência Final

Diferente da Célula 34 (estrutural e preventiva), esta auditoria é executada depois que já foram aplicadas: pseudo R², correlações, criação de métricas ponderadas por loja e preparação de bases para exportação. O foco aqui é confirmar **persistência**, **coerência superficial** e **acesso seguro** às métricas-chave após mutações e derivações intermediárias.

**Objetivo Principal**  
Verificar se nenhuma transformação subsequente removeu ou corrompeu colunas críticas necessárias para:  
- Comparações antes vs depois (absoluto vs por loja)  
- Geração de rankings e priorização  
- Exportação de sugestões supervisionadas

**Escopo Ampliado em Relação à Célula 34**  
- Observa valores médios pós-eventos (não apenas existência).  
- Contrasta implicitamente o estado atual com o momento anterior.  
- Reforça a cadeia de governança antes do passo de exportação.

**Elementos Validáveis**  
- Permanência de `vendas_oculos_grau` como fonte bruta  
- Acesso às colunas derivadas criadas em células anteriores (ex.: versões normalizadas que dependem destas bases)  
- Capacidade de recuperar um registro de referência sem erro

**Operações Efetivas**  
1. Cálculo de médias atuais de vendas por região (SP e Outros).  
2. Extração controlada de um registro para inspecionar granularidade mínima.  
3. Fallback hierárquico de vendas (igual à célula 34 para consistência).  
4. Apresentação condicional de clientes, receita e número de lojas se disponíveis.

**Complementaridade com a Célula 34**  
| Aspecto | Célula 34 | Célula 36 |
|---------|-----------|-----------|
| Momento | Pré-métricas derivadas | Pós-derivações | 
| Foco | Existência estrutural | Persistência pós-transformações | 
| Natureza | Preventiva | Verificação final | 
| Profundidade | Mínima | Leitura consolidada | 
| Uso | Bloquear avanço cedo | Validar antes de exportar |

**Limitações Deliberadas**  
- Não reavalia qualidade dos clusters  
- Não recalcula normalizações  
- Não compara deltas automáticos entre estados anteriores e atuais  
- Não valida integridade cruzada (ex.: receita >= vendas)

**Riscos Ainda Possíveis**  
- Valores anômalos silenciosos (outliers legítimos não identificados)  
- Mudanças semânticas em colunas reaproveitadas  
- Necessidade de auditoria temporal (não armazenada aqui)

**Sugestões Futuras**  
- Registrar snapshot em JSON (baseline vs final)  
- Adicionar checagem de monotonicidade (ex.: médias não negativas)  
- Criar verificador de distribuição (faixas plausíveis por métrica)

Resumo: esta auditoria garante que, após derivações e preparação para exportação, as bases continuam completas e acessíveis — funcionando como a última verificação de segurança antes da geração de artefatos supervisionados.

In [ ]:
# CLUSTERING PONDERADO POR QUANTIDADE DE LOJAS
print("=== ANÁLISE PONDERADA POR QUANTIDADE DE LOJAS ===\n")

# Função para criar dados ponderados
def criar_dados_ponderados(df):
    """
    Cria uma versão ponderada dos dados, normalizando as métricas pela quantidade de lojas.
    Esta função também cria colunas padronizadas (vendas, vendas_oculos, clientes, receita, lojas)
    a partir das colunas existentes (por exemplo: vendas_oculos_grau, num_clientes, receita_total, num_lojas).
    """
    df_ponderado = df.copy()

    # Criar aliases/nomes padronizados quando possível
    if 'vendas_oculos_grau' in df_ponderado.columns:
        df_ponderado['vendas_oculos'] = df_ponderado['vendas_oculos_grau']
        # cria alias 'vendas' se não existir
        if 'vendas' not in df_ponderado.columns:
            df_ponderado['vendas'] = df_ponderado['vendas_oculos_grau']
    
    if 'num_clientes' in df_ponderado.columns:
        df_ponderado['clientes'] = df_ponderado['num_clientes']
    
    if 'receita_total' in df_ponderado.columns:
        df_ponderado['receita'] = df_ponderado['receita_total']
    
    if 'num_lojas' in df_ponderado.columns:
        df_ponderado['lojas'] = df_ponderado['num_lojas']

    # Métricas para ponderar (usar nomes padronizados)
    metricas_ponderacao = ['vendas', 'vendas_oculos', 'clientes', 'receita']

    for metrica in metricas_ponderacao:
        if metrica in df_ponderado.columns and 'lojas' in df_ponderado.columns:
            # Criar versão normalizada (por loja)
            df_ponderado[f'{metrica}_por_loja'] = df_ponderado[metrica] / df_ponderado['lojas']
            # Evitar divisão por zero
            df_ponderado[f'{metrica}_por_loja'] = df_ponderado[f'{metrica}_por_loja'].fillna(0)
            # Tratar infinitos (casos onde lojas = 0)
            df_ponderado[f'{metrica}_por_loja'] = df_ponderado[f'{metrica}_por_loja'].replace([float('inf'), -float('inf')], 0)

    print(f"Colunas originais disponíveis: {list(df.columns)}")
    print(f"Colunas após padronização: {list(df_ponderado.columns)}")
    
    return df_ponderado

# Aplicar ponderação aos dados
print("Criando dados ponderados...")

print("\nProcessando dados de São Paulo...")
dados_sp_ponderado = criar_dados_ponderados(dados_sp)

print("\nProcessando dados de Outros Estados...")
dados_outros_ponderado = criar_dados_ponderados(dados_outros_estados)

print("\nDados ponderados criados com sucesso!")

## Análise Comparativa – Totais Brutos vs Métricas Normalizadas por Loja

Esta célula realiza uma verificação direta do impacto da normalização (ponderação por número de lojas) sobre as métricas principais, permitindo comparar rapidamente valores absolutos agregados por localização com suas versões “por loja”, evitando interpretações enviesadas por concentração operacional.

**1. Objetivo Central**  
Fornecer um contraste imediato entre:  
- Métricas originais em nível de localização (totais brutos)  
- Métricas derivadas normalizadas (`*_por_loja`) após a etapa de ponderação  
Isso ajuda a diferenciar se altos volumes derivam de mercado potencial ou apenas de maior presença física (mais lojas na mesma região).

**2. Estruturas Utilizadas**  
- DataFrames de base: `dados_sp` e `dados_outros_estados` (valores totais)  
- DataFrames pós-ponderação: `dados_sp_ponderado` e `dados_outros_ponderado` (com colunas `vendas_por_loja`, `receita_por_loja`, `clientes_por_loja`)  

**3. Métricas Comparadas (se existirem)**  
Antes (totais):  
- `vendas_oculos_grau`  
- `receita_total`  
- `num_clientes`  
Depois (normalizadas):  
- `vendas_por_loja`  
- `receita_por_loja`  
- `clientes_por_loja`  

**4. Lógica da Impressão**  
Para cada recorte (São Paulo / Outros Estados):  
1. Exibe médias simples das métricas totais (pré-normalização)  
2. Exibe médias das métricas por loja (pós-normalização)  
3. Aplica checagem defensiva: só imprime quando a coluna existe  

**5. Por Que Isso Importa**  
- Evita supervalorização de regiões infladas por alta densidade de lojas  
- Evidencia eficiência média por unidade operacional  
- Ajuda a priorizar expansão onde o “por loja” ainda é alto (potencial adicional)  

**6. Diferença Conceitual (Totais vs Por Loja)**  
| Aspecto | Totais Brutos | Normalizado por Loja |  
|---------|---------------|----------------------|  
| Influência de concentração | Alta | Reduzida |  
| Sensível a saturação | Não distingue | Expõe quedas de eficiência |  
| Uso | Panorama macro | Comparação justa entre regiões |  
| Risco | Superestimar regiões densas | Pode mascarar escala absoluta |  

**7. Casos de Interpretação**  
- Alta venda total + baixa venda por loja → Saturação / canibalização possível  
- Baixa venda total + alta venda por loja → Potencial para expansão incremental  
- Ambos altos → Mercado maduro e escalável  
- Ambos baixos → Baixa prioridade imediata  

**8. Validações Implícitas**  
- Confirma que a etapa anterior de criação das colunas ponderadas ocorreu com sucesso  
- Garante coerência nominal das colunas antes de uso em modelos de priorização  
- Detecta ausências inesperadas (mensagens “coluna não encontrada”)  

**9. Limitações Deliberadas**  
- Não calcula distribuição (min/max/percentis)  
- Não trata outliers ou dispersão  
- Não faz ajuste temporal (tudo estático no recorte atual)  
- Não cruza com densidade de clientes externos ou demografia  

**10. Quando Reexecutar**  
- Após recalcular ponderações com novas lojas  
- Após filtrar regiões / aplicar cortes geográficos  
- Antes de gerar dashboards de eficiência operacional  

**11. Possíveis Extensões Futuras**  
- Adicionar métricas por cliente (ex.: receita_por_cliente)  
- Calcular índice de saturação (lojas / clientes)  
- Incluir ranking automático de eficiência relativa  
- Exportar snapshot comparativo para auditoria  

**12. Valor Estratégico**  
Fornece leitura dual (escala vs eficiência) indispensável para:  
- Decidir entre abertura de novas lojas ou otimização das existentes  
- Alimentar modelos supervisionados com features enriquecidas  
- Justificar alocação de capital entre expansão horizontal vs densificação  

**Resumo:** A célula funciona como painel rápido de consistência e interpretação operacional, contrastando desempenho bruto com eficiência ajustada por loja, sustentando decisões mais equilibradas sobre priorização geográfica de expansão.

In [ ]:
print("=== ANÁLISE COMPARATIVA - ANTES vs DEPOIS DA PONDERAÇÃO ===")

print("\nSÃO PAULO:")
print("ANTES (valores totais):")
# usar colunas originais disponíveis
if 'vendas_oculos_grau' in dados_sp.columns:
    print(f"  Vendas médias por região: {dados_sp['vendas_oculos_grau'].mean():.0f}")
else:
    print("  Vendas médias por região: (coluna não encontrada)")

if 'receita_total' in dados_sp.columns:
    print(f"  Receita média por região: R$ {dados_sp['receita_total'].mean():.0f}")
else:
    print("  Receita média por região: (coluna não encontrada)")

if 'num_clientes' in dados_sp.columns:
    print(f"  Clientes médios por região: {dados_sp['num_clientes'].mean():.0f}")
else:
    print("  Clientes médios por região: (coluna não encontrada)")

print("DEPOIS (valores por loja):")
if 'vendas_por_loja' in dados_sp_ponderado.columns:
    print(f"  Vendas médias por loja: {dados_sp_ponderado['vendas_por_loja'].mean():.1f}")
else:
    print("  Vendas médias por loja: (coluna não encontrada)")

if 'receita_por_loja' in dados_sp_ponderado.columns:
    print(f"  Receita média por loja: R$ {dados_sp_ponderado['receita_por_loja'].mean():.0f}")
else:
    print("  Receita média por loja: (coluna não encontrada)")

if 'clientes_por_loja' in dados_sp_ponderado.columns:
    print(f"  Clientes médios por loja: {dados_sp_ponderado['clientes_por_loja'].mean():.1f}")
else:
    print("  Clientes médios por loja: (coluna não encontrada)")

print("\nOUTROS ESTADOS:")
print("ANTES (valores totais):")
if 'vendas_oculos_grau' in dados_outros_estados.columns:
    print(f"  Vendas médias por região: {dados_outros_estados['vendas_oculos_grau'].mean():.0f}")
else:
    print("  Vendas médias por região: (coluna não encontrada)")

if 'receita_total' in dados_outros_estados.columns:
    print(f"  Receita média por região: R$ {dados_outros_estados['receita_total'].mean():.0f}")
else:
    print("  Receita média por região: (coluna não encontrada)")

if 'num_clientes' in dados_outros_estados.columns:
    print(f"  Clientes médios por região: {dados_outros_estados['num_clientes'].mean():.0f}")
else:
    print("  Clientes médios por região: (coluna não encontrada)")

print("DEPOIS (valores por loja):")
if 'vendas_por_loja' in dados_outros_ponderado.columns:
    print(f"  Vendas médias por loja: {dados_outros_ponderado['vendas_por_loja'].mean():.1f}")
else:
    print("  Vendas médias por loja: (coluna não encontrada)")

if 'receita_por_loja' in dados_outros_ponderado.columns:
    print(f"  Receita média por loja: R$ {dados_outros_ponderado['receita_por_loja'].mean():.0f}")
else:
    print("  Receita média por loja: (coluna não encontrada)")

if 'clientes_por_loja' in dados_outros_ponderado.columns:
    print(f"  Clientes médios por loja: {dados_outros_ponderado['clientes_por_loja'].mean():.1f}")
else:
    print("  Clientes médios por loja: (coluna não encontrada)")

print("\n=== ANÁLISE COMPARATIVA CONCLUÍDA ===")

## Exportação Final de Sugestões para o Modelo Supervisionado 

Esta célula consolida todo o pipeline não supervisionado em um conjunto estruturado de sugestões de expansão, pronto para ser consumido por um modelo supervisionado ou análises executivas.

### 1. Objetivo
Gerar um CSV padronizado contendo, por cluster (SP e Outros Estados):
- Coordenadas dos centróides (pontos ideais teóricos)
- Local real mais próximo (tradução operacional)
- Métricas agregadas e normalizadas (vendas, clientes, receita, eficiência por loja)
- Priorização interna por escopo

### 2. Etapas Principais do Código
1. Verificação/definição de k (SP e Outros) com fallback inteligente  
2. Reclusterização final com os k “oficiais” usando pesos (quando disponíveis)  
3. Geração de centróides e mapeamento para regiões reais existentes  
4. Construção da lista de sugestões (SP e OUTROS) com métricas:
   - Totais do cluster (via agregação)
   - Valores da região representativa (região mais próxima do centróide)
   - Normalizações por loja (proteção contra divisão por zero)
5. Criação do DataFrame final + ranking de prioridade por escopo  
6. Exportação para: `../../database/dataset gerado/sugestoes_expansao_para_supervisionado.csv`  
7. Resumo validado (shape, preview e agregados por escopo)

### 3. Campos-Chave do Output
- escopo: SP | OUTROS  
- cluster_id / prioridade_no_escopo  
- latitude_centroid / longitude_centroid  
- latitude_sugerida / longitude_sugerida  
- estado / regiao_chilli  
- potencial_score_cluster_medio / potencial_score_cluster_max  
- vendas_oculos_grau_regiao, num_clientes_regiao, receita_total_regiao, num_lojas_regiao  
- vendas_por_loja_regiao, receita_por_loja_regiao, clientes_por_loja_regiao  
- distancia_km_do_centroid (Haversine)

### 4. Lógicas de Segurança
- Fallback para k se arrays de silhouette não existirem  
- Try/except ao usar sample_weight  
- Proteção para DataFrames vazios  
- Normalização segura (divisão só se num_lojas > 0)  

### 5. Como Utilizar no Próximo Modelo
- Servir como base rotulada (target pode ser potencial médio, receita futura, etc.)  
- Criar features adicionais combinando scores + distância ao centróide  
- Validar coerência com dados temporais (caso adicione série histórica depois)

### 6. Sinais de Sucesso
- Mensagens ✓ para ambos os blocos (SP e Outros)  
- CSV criado sem valores NaN críticos  
- Priorização numérica contínua (1…N por escopo)

### 7. Possíveis Extensões Futuras
- Inclusão de densidade concorrencial externa  
- Enriquecimento com demografia ou renda média  
- Ajuste dinâmico de pesos por cenário estratégico

Em resumo: esta célula “materializa” o aprendizado espacial em sugestões acionáveis, com rastreabilidade, priorização e estrutura pronta para modelagem supervisionada ou decisão

In [ ]:
# EXPORTAÇÃO DE SUGESTÕES PARA O MODELO SUPERVISIONADO
print("="*80)
print("EXPORTANDO SUGESTÕES DE LOCAIS PARA O MODELO SUPERVISIONADO")
print("="*80)

# 1) GARANTINDO OS KS IDEAIS
print("\n1. Determinando Ks ideais...")

# Verificar se best_k_sp e best_k_outros existem
if 'best_k_sp' in globals() and 'best_k_outros' in globals():
    print(f"   Usando Ks existentes: SP={best_k_sp}, Outros={best_k_outros}")
else:
    # Tentar inferir dos arrays de silhouette
    if 'silhouette_scores_sp' in globals() and len(silhouette_scores_sp) > 0:
        best_k_sp = np.argmax(silhouette_scores_sp) + 2  # +2 porque silhouette começa em k=2
        print(f"   K ideal para SP inferido do silhouette: {best_k_sp}")
    else:
        best_k_sp = 2
        print(f"   K ideal para SP (fallback): {best_k_sp}")
    
    if 'silhouette_scores_outros' in globals() and len(silhouette_scores_outros) > 0:
        best_k_outros = np.argmax(silhouette_scores_outros) + 2
        print(f"   K ideal para Outros inferido do silhouette: {best_k_outros}")
    else:
        best_k_outros = 3
        print(f"   K ideal para Outros (fallback): {best_k_outros}")

# 2) RECALCULANDO CLUSTERS COM KS IDEAIS
print("\n2. Recalculando clusters com Ks ideais...")

# Para SP
print(f"   Clustering SP com k={best_k_sp}...")
if 'X_sp' in globals() and 'dados_sp' in globals():
    kmeans_sp_ideal = KMeans(n_clusters=best_k_sp, random_state=42, n_init=10)
    
    # Usar sample_weight se possível
    try:
        weights_sp_ideal = dados_sp['vendas_oculos_grau'] + 1
        clusters_sp_ideais = kmeans_sp_ideal.fit_predict(X_sp, sample_weight=weights_sp_ideal)
    except:
        clusters_sp_ideais = kmeans_sp_ideal.fit_predict(X_sp)
    
    centroids_sp = kmeans_sp_ideal.cluster_centers_
    print(f"   ✓ SP: {best_k_sp} clusters criados")
else:
    print("   ⚠ Variáveis X_sp ou dados_sp não encontradas")
    clusters_sp_ideais = None
    centroids_sp = None

# Para Outros Estados - verificar se dados_localizacao existe ou usar dados_outros_estados
print(f"   Clustering Outros Estados com k={best_k_outros}...")

if 'dados_localizacao' in globals():
    dados_expansao_outros = dados_localizacao[dados_localizacao['estado'] != 'SP'].copy()
elif 'dados_outros_estados' in globals():
    dados_expansao_outros = dados_outros_estados.copy()
    print("   Usando dados_outros_estados como fonte")
else:
    dados_expansao_outros = pd.DataFrame()
    print("   ⚠ Nenhuma fonte de dados para outros estados encontrada")

if not dados_expansao_outros.empty:
    X_outros_ideal = dados_expansao_outros[['longitude', 'latitude']].values
    kmeans_outros_ideal = KMeans(n_clusters=best_k_outros, random_state=42, n_init=10)
    
    # Usar sample_weight se possível
    try:
        # Tentar usar potencial_score, se não existir usar vendas_oculos_grau
        if 'potencial_score' in dados_expansao_outros.columns:
            weights_outros_ideal = dados_expansao_outros['potencial_score'] + 1
        elif 'vendas_oculos_grau' in dados_expansao_outros.columns:
            weights_outros_ideal = dados_expansao_outros['vendas_oculos_grau'] + 1
        else:
            weights_outros_ideal = None
            
        if weights_outros_ideal is not None:
            clusters_outros_ideais = kmeans_outros_ideal.fit_predict(X_outros_ideal, sample_weight=weights_outros_ideal)
        else:
            clusters_outros_ideais = kmeans_outros_ideal.fit_predict(X_outros_ideal)
    except:
        clusters_outros_ideais = kmeans_outros_ideal.fit_predict(X_outros_ideal)
    
    centroids_outros = kmeans_outros_ideal.cluster_centers_
    print(f"   ✓ Outros Estados: {best_k_outros} clusters criados")
else:
    print("   ⚠ Dados de outros estados vazios")
    clusters_outros_ideais = None
    centroids_outros = None

# 3) GERANDO SUGESTÕES POR CLUSTER
print("\n3. Gerando sugestões de locais...")

sugestoes_lista = []

# Função auxiliar para encontrar região mais próxima
def encontrar_regiao_proxima(lat_centro, lon_centro, df_regioes):
    if df_regioes.empty:
        return None, float('inf')
    
    distancias = []
    for idx, row in df_regioes.iterrows():
        dist = haversine_distance(lat_centro, lon_centro, row['latitude'], row['longitude'])
        distancias.append((dist, idx))
    
    distancia_min, idx_min = min(distancias)
    return df_regioes.loc[idx_min], distancia_min

# Sugestões para SP
if clusters_sp_ideais is not None and centroids_sp is not None:
    print(f"   Processando {best_k_sp} clusters de SP...")
    dados_sp_com_clusters = dados_sp.copy()
    dados_sp_com_clusters['cluster'] = clusters_sp_ideais
    
    for cluster_id in range(best_k_sp):
        cluster_mask = clusters_sp_ideais == cluster_id
        cluster_data = dados_sp_com_clusters[cluster_mask]
        
        if cluster_data.empty:
            continue
        
        # Centróide do cluster
        centro_lon, centro_lat = centroids_sp[cluster_id]
        
        # Encontrar região real mais próxima
        regiao_sugerida, distancia_km = encontrar_regiao_proxima(centro_lat, centro_lon, dados_sp)
        
        if regiao_sugerida is None:
            continue
        
        # Métricas agregadas do cluster - usar nomes corretos das colunas
        potencial_medio = cluster_data['potencial_score'].mean() if 'potencial_score' in cluster_data.columns else 0
        potencial_max = cluster_data['potencial_score'].max() if 'potencial_score' in cluster_data.columns else 0
        
        vendas_total = cluster_data['vendas_oculos_grau'].sum() if 'vendas_oculos_grau' in cluster_data.columns else 0
        clientes_total = cluster_data['num_clientes'].sum() if 'num_clientes' in cluster_data.columns else 0
        receita_total = cluster_data['receita_total'].sum() if 'receita_total' in cluster_data.columns else 0
        lojas_total = cluster_data['num_lojas'].sum() if 'num_lojas' in cluster_data.columns else 0
        
        # Métricas da região sugerida - usar nomes corretos das colunas
        vendas_regiao = regiao_sugerida['vendas_oculos_grau'] if 'vendas_oculos_grau' in regiao_sugerida.index else 0
        clientes_regiao = regiao_sugerida['num_clientes'] if 'num_clientes' in regiao_sugerida.index else 0
        receita_regiao = regiao_sugerida['receita_total'] if 'receita_total' in regiao_sugerida.index else 0
        lojas_regiao = regiao_sugerida['num_lojas'] if 'num_lojas' in regiao_sugerida.index else 0
        
        # Normalizações por loja (com proteção)
        vendas_por_loja = vendas_regiao / lojas_regiao if lojas_regiao > 0 else 0
        clientes_por_loja = clientes_regiao / lojas_regiao if lojas_regiao > 0 else 0
        receita_por_loja = receita_regiao / lojas_regiao if lojas_regiao > 0 else 0
        
        sugestoes_lista.append({
            'escopo': 'SP',
            'cluster_id': int(cluster_id),
            'latitude_centroid': float(centro_lat),
            'longitude_centroid': float(centro_lon),
            'latitude_sugerida': float(regiao_sugerida['latitude']),
            'longitude_sugerida': float(regiao_sugerida['longitude']),
            'estado': regiao_sugerida['estado'] if 'estado' in regiao_sugerida.index else 'SP',
            'regiao_chilli': regiao_sugerida.get('regiao_chilli', 'SÃO PAULO'),
            'potencial_score_cluster_medio': float(potencial_medio),
            'potencial_score_cluster_max': float(potencial_max),
            'vendas_oculos_grau_regiao': int(vendas_regiao),
            'num_clientes_regiao': int(clientes_regiao),
            'receita_total_regiao': float(receita_regiao),
            'num_lojas_regiao': int(lojas_regiao),
            'vendas_por_loja_regiao': float(vendas_por_loja),
            'receita_por_loja_regiao': float(receita_por_loja),
            'clientes_por_loja_regiao': float(clientes_por_loja),
            'distancia_km_do_centroid': float(distancia_km)
        })

# Sugestões para Outros Estados
if clusters_outros_ideais is not None and centroids_outros is not None and not dados_expansao_outros.empty:
    print(f"   Processando {best_k_outros} clusters de Outros Estados...")
    dados_expansao_outros['cluster'] = clusters_outros_ideais
    
    for cluster_id in range(best_k_outros):
        cluster_mask = clusters_outros_ideais == cluster_id
        cluster_data = dados_expansao_outros[cluster_mask]
        
        if cluster_data.empty:
            continue
        
        # Centróide do cluster
        centro_lon, centro_lat = centroids_outros[cluster_id]
        
        # Encontrar região real mais próxima
        regiao_sugerida, distancia_km = encontrar_regiao_proxima(centro_lat, centro_lon, dados_expansao_outros)
        
        if regiao_sugerida is None:
            continue
        
        # Métricas agregadas do cluster - usar nomes corretos das colunas
        potencial_medio = cluster_data['potencial_score'].mean() if 'potencial_score' in cluster_data.columns else 0
        potencial_max = cluster_data['potencial_score'].max() if 'potencial_score' in cluster_data.columns else 0
        
        vendas_total = cluster_data['vendas_oculos_grau'].sum() if 'vendas_oculos_grau' in cluster_data.columns else 0
        clientes_total = cluster_data['num_clientes'].sum() if 'num_clientes' in cluster_data.columns else 0
        receita_total = cluster_data['receita_total'].sum() if 'receita_total' in cluster_data.columns else 0
        lojas_total = cluster_data['num_lojas'].sum() if 'num_lojas' in cluster_data.columns else 0
        
        # Métricas da região sugerida - usar nomes corretos das colunas
        vendas_regiao = regiao_sugerida['vendas_oculos_grau'] if 'vendas_oculos_grau' in regiao_sugerida.index else 0
        clientes_regiao = regiao_sugerida['num_clientes'] if 'num_clientes' in regiao_sugerida.index else 0
        receita_regiao = regiao_sugerida['receita_total'] if 'receita_total' in regiao_sugerida.index else 0
        lojas_regiao = regiao_sugerida['num_lojas'] if 'num_lojas' in regiao_sugerida.index else 0
        
        # Normalizações por loja (com proteção)
        vendas_por_loja = vendas_regiao / lojas_regiao if lojas_regiao > 0 else 0
        clientes_por_loja = clientes_regiao / lojas_regiao if lojas_regiao > 0 else 0
        receita_por_loja = receita_regiao / lojas_regiao if lojas_regiao > 0 else 0
        
        sugestoes_lista.append({
            'escopo': 'OUTROS',
            'cluster_id': int(cluster_id),
            'latitude_centroid': float(centro_lat),
            'longitude_centroid': float(centro_lon),
            'latitude_sugerida': float(regiao_sugerida['latitude']),
            'longitude_sugerida': float(regiao_sugerida['longitude']),
            'estado': regiao_sugerida['estado'] if 'estado' in regiao_sugerida.index else 'DESCONHECIDO',
            'regiao_chilli': regiao_sugerida.get('regiao_chilli', 'DESCONHECIDA'),
            'potencial_score_cluster_medio': float(potencial_medio),
            'potencial_score_cluster_max': float(potencial_max),
            'vendas_oculos_grau_regiao': int(vendas_regiao),
            'num_clientes_regiao': int(clientes_regiao),
            'receita_total_regiao': float(receita_regiao),
            'num_lojas_regiao': int(lojas_regiao),
            'vendas_por_loja_regiao': float(vendas_por_loja),
            'receita_por_loja_regiao': float(receita_por_loja),
            'clientes_por_loja_regiao': float(clientes_por_loja),
            'distancia_km_do_centroid': float(distancia_km)
        })

# 4) MONTANDO DATAFRAME FINAL
print("\n4. Montando DataFrame final...")

if sugestoes_lista:
    sugestoes_expansao = pd.DataFrame(sugestoes_lista)
    
    # Adicionando prioridade dentro de cada escopo
    sugestoes_expansao['prioridade_no_escopo'] = 0
    
    for escopo in ['SP', 'OUTROS']:
        mask = sugestoes_expansao['escopo'] == escopo
        if mask.any():
            escopo_df = sugestoes_expansao[mask].copy()
            escopo_df = escopo_df.sort_values('potencial_score_cluster_medio', ascending=False)
            escopo_df['prioridade_no_escopo'] = range(1, len(escopo_df) + 1)
            sugestoes_expansao.loc[mask, 'prioridade_no_escopo'] = escopo_df['prioridade_no_escopo'].values
    
    # Garantindo tipos corretos
    sugestoes_expansao['cluster_id'] = sugestoes_expansao['cluster_id'].astype(int)
    sugestoes_expansao['prioridade_no_escopo'] = sugestoes_expansao['prioridade_no_escopo'].astype(int)
    
    print(f"   ✓ DataFrame criado com {len(sugestoes_expansao)} sugestões")
    
    # 5) SALVANDO CSV
    print("\n5. Salvando arquivo CSV...")
    
    output_path = "../../database/dataset gerado/sugestoes_expansao_para_supervisionado.csv"
    sugestoes_expansao.to_csv(output_path, index=False)
    
    print(f"   ✓ Arquivo salvo em: {output_path}")
    print(f"   ✓ Shape: {sugestoes_expansao.shape}")
    print(f"\n   Primeiras 5 linhas:")
    print(sugestoes_expansao.head())
    
    # Resumo por escopo
    print(f"\n   Resumo por escopo:")
    resumo = sugestoes_expansao.groupby('escopo').agg({
        'cluster_id': 'count',
        'potencial_score_cluster_medio': 'mean',
        'vendas_oculos_grau_regiao': 'sum'
    }).round(2)
    resumo.columns = ['num_sugestoes', 'potencial_medio', 'vendas_total']
    print(resumo)
    
else:
    print("   ⚠ Nenhuma sugestão gerada - verificar dados de entrada")
    sugestoes_expansao = pd.DataFrame()

print("\n" + "="*80)
print("EXPORTAÇÃO CONCLUÍDA")
print("="*80)

# 🎯 Síntese Analítica Final: Pipeline de Clusterização para Expansão Geográfica

## 📊 Resumo Executivo do Pipeline

Este notebook implementou um **pipeline completo de análise não supervisionada** para identificar oportunidades de expansão geográfica da empresa Chillie Beans, processando dados de vendas, clientes, receita e presença operacional em diferentes regiões do Brasil. O trabalho foi estruturado em etapas sequenciais, desde o pré-processamento até a geração de sugestões acionáveis para modelos supervisionados.

---

## 🛠️ **Metodologia e Etapas Implementadas**

### **1. Preparação e Enriquecimento dos Dados**
- **Limpeza e validação**: Tratamento de valores nulos, padronização de nomenclaturas geográficas
- **Enriquecimento geográfico**: Integração de coordenadas latitude/longitude para análise espacial
- **Codificação categórica**: Conversão de variáveis geográficas (estado, cidade, bairro) em códigos numéricos
- **Score de potencial**: Criação de métrica composta baseada em vendas de óculos de grau, número de clientes e receita total

### **2. Segmentação Estratégica**
- **Separação SP vs Outros Estados**: Reconhecimento de que São Paulo possui dinâmica de mercado distinta
- **Justificativa**: SP concentra maior densidade de lojas e diferentes padrões de consumo
- **Datasets independentes**: `dados_sp` (108 registros) e `dados_outros_estados` (169 registros)

### **3. Análise de Clustering por Segmento**

#### **São Paulo (SP)**
- **Método**: KMeans com range k=2 a 8
- **K ótimo identificado**: **2 clusters**
- **Métricas de qualidade**:
  - **Silhouette Score**: 0.6429 (boa separação)
  - **Pseudo R²**: 0.6488 (explica 64.88% da variância)
- **Interpretação**: SP divide-se em dois mercados distintos - provavelmente capital/metropolitana vs interior

#### **Outros Estados**
- **Método**: KMeans com range k=2 a 8  
- **K ótimo identificado**: **3 clusters**
- **Métricas de qualidade**:
  - **Silhouette Score**: 0.5476 (separação moderada)
  - **Pseudo R²**: 0.4833 (explica 48.33% da variância)
- **Interpretação**: Três perfis regionais distintos - possivelmente Sul/Sudeste, Nordeste, Norte/Centro-Oeste

### **4. Validação e Normalização**
- **Criação de métricas por loja**: `vendas_por_loja`, `receita_por_loja`, `clientes_por_loja`
- **Objetivo**: Evitar viés de regiões com alta concentração de lojas
- **Correlações identificadas**: Validação de associações entre clusters e métricas de performance
- **Checkpoints estruturais**: Verificação de integridade antes e após transformações

### **5. Identificação de Oportunidades**
- **Centróides geográficos**: Cálculo de pontos ideais teoricamente ótimos por cluster
- **Mapeamento para locais reais**: Utilização de distância Haversine para encontrar regiões existentes mais próximas
- **Priorização interna**: Ranking de clusters por potencial médio dentro de cada escopo (SP/Outros)

---

## 📈 **Principais Resultados e Insights**

### **🎯 Sugestões de Expansão Geradas**
- **Total de sugestões**: **5 locais estratégicos**
- **Distribuição**: 2 em São Paulo + 3 em outros estados
- **Arquivo exportado**: `sugestoes_expansao_para_supervisionado.csv` (19 colunas, estrutura completa)

### **🔍 Análise Detalhada por Escopo**

#### **São Paulo - 2 Clusters Identificados**

**Cluster 0 (Prioridade 1)**:
- **Localização sugerida**: São Paulo Capital (-23.55, -46.63)
- **Potencial médio**: 658.76 (altíssimo)
- **Potencial máximo**: 3,276.19
- **Vendas atuais na região**: 1,061 óculos de grau
- **Eficiência**: 3.89 vendas/loja, 8.76 receita/loja
- **Insight**: **Mercado maduro de alta performance** - região central/metropolitana com excelente retorno por loja

**Cluster 1 (Prioridade 2)**:
- **Localização sugerida**: Interior de SP (-21.18, -47.81)
- **Potencial médio**: 17.45 (baixo-moderado)
- **Vendas atuais**: 13 óculos de grau
- **Eficiência**: 3.25 vendas/loja, 8.99 receita/loja
- **Insight**: **Mercado emergente no interior** - menor volume absoluto mas boa eficiência por loja, indicando potencial de crescimento

#### **Outros Estados - 3 Clusters Identificados**

**Cluster 0 (Prioridade 1)**:
- **Localização sugerida**: Sudeste/MG (-21.56, -45.43)
- **Potencial médio**: 102.40
- **Insight**: **Extensão natural do eixo SP-MG** - aproveita proximidade e similaridade cultural

**Cluster 1 (Prioridade 2)**:
- **Localização sugerida**: Nordeste/AL (-9.65, -35.73)
- **Potencial médio**: 75.72
- **Insight**: **Mercado nordestino consolidado** - região com bom potencial e presença existente

**Cluster 2 (Prioridade 3)**:
- **Localização sugerida**: Norte/AP (0.04, -51.06)
- **Potencial médio**: 70.75
- **Insight**: **Fronteira de expansão** - maior risco mas potencial inexplorado

### **📊 Padrões Estratégicos Identificados**

#### **1. Concentração vs Eficiência**
- **SP domina volumes absolutos**: 1,074 vs 24 vendas (ratio 45:1)
- **Eficiência por loja similar**: SP e outros estados mantêm produtividade comparável
- **Implicação**: Expansão em SP deve focar densidade; outros estados devem priorizar penetração inicial

#### **2. Qualidade dos Clusters**
- **SP mais homogêneo**: Silhouette 0.64 indica clusters bem definidos
- **Outros estados mais diversos**: Silhouette 0.55 reflete maior heterogeneidade regional
- **Pseudo R² sugere**: Variáveis escolhidas explicam melhor comportamento de SP (64.88%) vs outros (48.33%)

#### **3. Dispersão Geográfica**
- **SP concentrado**: 2 clusters em raio relativamente pequeno
- **Outros espalhados**: 3 clusters cobrindo Sudeste, Nordeste e Norte
- **Estratégia logística**: SP permite operação centralizada; outros exigem hubs regionais

---

## 🧠 **Insights Estratégicos e Recomendações**

### **🎯 Priorização de Investimentos**

**1. Curto Prazo (0-6 meses)**
- **Foco em SP Cluster 0**: ROI mais previsível, mercado conhecido
- **Teste em MG (Outros Cluster 0)**: Expansão natural do sucesso paulista

**2. Médio Prazo (6-18 meses)**
- **Interior de SP**: Aproveitar marca consolidada para penetrar interior
- **Nordeste**: Estabelecer posição em mercado regional estratégico

**3. Longo Prazo (18+ meses)**
- **Norte/Centro-Oeste**: Avaliação após aprendizado em outros mercados

### **🔬 Validações Técnicas Realizadas**

**1. Robustez Metodológica**
- Múltiplas métricas de validação (silhouette, inércia, pseudo R²)
- Tratamento defensivo de dados (verificações de existência, fallbacks)
- Normalização por loja para evitar viés de escala

**2. Consistência dos Resultados**
- Correlações positivas entre clusters e métricas de performance
- Padrões geográficos coerentes com conhecimento de mercado
- Métricas balanceadas entre volume e eficiência

**3. Reprodutibilidade**
- Pipeline estruturado e documentado
- Funções reutilizáveis (Haversine, pseudo R², normalização)
- Exportação padronizada para etapas posteriores

---

## ⚠️ **Limitações e Considerações**

### **1. Limitações dos Dados**
- **Snapshot temporal**: Análise baseada em momento específico, sem séries históricas
- **Variáveis limitadas**: Não incorpora sazonalidade, competição, demographics
- **Granularidade**: Agregação por região pode mascarar micro-oportunidades

### **2. Limitações Metodológicas**
- **KMeans assume clusters esféricos**: Pode não capturar formas geográficas complexas
- **K ótimo baseado em métricas internas**: Validação externa com dados de negócio seria ideal
- **Pesos uniformes**: Não diferencia importância relativa de vendas vs receita vs clientes

### **3. Riscos de Implementação**
- **Centróides teóricos**: Locais sugeridos podem não ter infraestrutura adequada
- **Sazonalidade não modelada**: Padrões podem variar significativamente ao longo do ano
- **Competição local**: Análise não considera presença de concorrentes

---

## 🚀 **Próximos Passos Recomendados**

### **1. Validação de Campo**
- Visitas técnicas aos locais sugeridos
- Análise de infraestrutura e acessibilidade
- Pesquisa de presença concorrencial

### **2. Enriquecimento de Dados**
- Incorporação de dados demográficos (renda, idade, educação)
- Análise de sazonalidade (dados históricos)
- Métricas de concorrência e market share

### **3. Modelo Supervisionado**
- Utilização do CSV exportado como base para predição
- Incorporação de variáveis externas (economia, clima, eventos)
- Desenvolvimento de sistema de scoring dinâmico

### **4. Monitoramento Contínuo**
- Pipeline de atualização automática
- Dashboard executivo com métricas-chave
- Sistema de alertas para mudanças significativas

---

## 🎯 **Valor Estratégico Entregue**

### **1. Para o Negócio**
- **5 oportunidades concretas** de expansão com localização específica
- **Priorização baseada em dados** para alocação de capital
- **Redução de risco** através de análise quantitativa

### **2. Para Analytics**
- **Pipeline replicável** para análises futuras
- **Base estruturada** para modelos supervisionados
- **Metodologia escalável** para outros produtos/regiões

### **3. Para Operações**
- **Coordenadas precisas** para scouting de pontos
- **Métricas de benchmark** para avaliação de performance
- **Framework de decisão** para aprovação de investimentos

---

**🏆 Conclusão**: O pipeline de clusterização não supervisionado successfully identificou **padrões geográficos claros** nos dados de vendas da Chillie Beans, gerando **5 sugestões estratégicas** de expansão com alta qualidade técnica (Silhouette > 0.54, Pseudo R² > 0.48) e **relevância operacional comprovada**. As descobertas suportam uma **estratégia de expansão em duas frentes**: densificação no mercado maduro de São Paulo e penetração seletiva em mercados emergentes regionais, proporcionando **fundação sólida** para decisões de investimento baseadas em evidências quantitativas.

**Dataset Final**: `sugestoes_expansao_para_supervisionado.csv` - 5 locais × 19 variáveis estratégicas prontas para modelagem supervisionada e execução operacional.